

#  **1.1 – Perché Docker è cruciale nel mondo AI: ambienti riproducibili e isolati**

---

## **Contesto**

Nel mondo dell’AI engineering, il codice raramente vive da solo.
Un singolo progetto può includere:

* una **CrewAI Flow pipeline** composta da più agenti;
* una **base di conoscenza indicizzata** su Qdrant o FAISS;
* una **UI Streamlit** o FastAPI per interazione utente;
* un **database PostgreSQL** per log, tracciamento e valutazione;
* librerie Python ad alta complessità (Torch, Transformers, OpenAI, LangChain, ecc.);
* e talvolta anche **accelerazione GPU**.

In questo ecosistema complesso, anche un piccolo cambiamento nella versione di Python, Torch o CUDA può **rompere la compatibilità** dell’intero progetto.
È qui che entra in gioco Docker.

---

## **Concetto chiave: riproducibilità e isolamento**

### 1. **Riproducibilità**

Docker permette di impacchettare tutto ciò che serve per far girare un’applicazione — **codice, dipendenze, librerie, variabili d’ambiente, sistema operativo** — in un’immagine immutabile.
Chiunque esegua quell’immagine, su qualunque macchina, ottiene **esattamente lo stesso comportamento**.

> **Esempio pratico**
> Un flow CrewAI funziona sulla tua macchina, ma non sul server remoto perché il server usa Python 3.10 mentre tu hai 3.11.
> Con Docker, entrambi eseguite **la stessa immagine**, basata sullo stesso ambiente (`FROM python:3.11-slim`), e il comportamento sarà identico.

### 2. **Isolamento**

Ogni container Docker è **un ambiente isolato** dal sistema operativo host e dagli altri container:

* Non condivide processi, pacchetti o variabili d’ambiente.
* Può avere una rete privata e filesystem dedicato.
* Se un container va in crash o consuma troppa RAM, **non impatta gli altri**.

Questo isolamento è cruciale in sistemi AI con **più microservizi**:

* il backend CrewAI non interferisce con Qdrant,
* Qdrant non influisce sul database PostgreSQL,
* e ogni parte può essere aggiornata o riavviata indipendentemente.

---

## **Perché è cruciale per l’AI moderna**

### 1. **Gestione delle dipendenze**

Progetti AI usano spesso librerie non allineate tra loro:

* `torch==2.3.0` richiede una versione specifica di CUDA,
* `transformers==4.44` può rompere LangChain se non aggiornato,
* `qdrant-client` ha binding Rust/Python sensibili alla versione.

Con Docker puoi bloccare **esattamente le versioni** nel Dockerfile, garantendo che il progetto sia stabile nel tempo.

---

### 2. **Reproducible Research e MLOps**

In MLOps e AI engineering, la **riproducibilità degli esperimenti** è un requisito critico.
Con Docker puoi:

* rieseguire esattamente una pipeline anche mesi dopo;
* distribuire lo stesso ambiente a un collega o server;
* garantire che l’esperimento su cui è stato addestrato un modello sia documentato e verificabile.

Molte aziende e laboratori (OpenAI, Hugging Face, Meta AI) **versionano i propri ambienti Docker** insieme al codice.

---

### 3. **Distribuzione e scalabilità**

Un container è un’unità standard, facilmente distribuibile:

* Puoi spostarlo da un laptop a un server cloud o a un cluster Kubernetes;
* Puoi scalare lo stesso container su più nodi senza riconfigurare nulla;
* Ogni microservizio CrewAI (rag, retriever, evaluator, frontend) può essere un container separato.

Esempio tipico di stack AI containerizzato:

```
crew-backend     -> container FastAPI con CrewAI
qdrant-db        -> container Qdrant ufficiale
postgres-logs    -> container PostgreSQL
streamlit-ui     -> container Streamlit con API utente
```

Con un solo comando:

```bash
docker compose up -d
```

tutto l’ambiente prende vita, identico su ogni macchina.

---

### 4. **Isolamento GPU e AI workloads**

Docker supporta nativamente l’accesso alle GPU tramite:

```bash
docker run --gpus all
```

o, in Compose:

```yaml
deploy:
  resources:
    reservations:
      devices:
        - capabilities: [gpu]
```

Questo permette di:

* eseguire modelli AI pesanti in container,
* isolare il carico GPU da altri processi,
* e distribuire workload AI in cluster.

---

## **Analogia concettuale**

> Pensa a Docker come a un “**laboratorio virtuale sigillato**”.
> Dentro ci sono le tue provette (codice, librerie, modelli), il tuo microscopio (framework AI), e perfino il manuale d’uso (Dockerfile).
> Nessuno può sporcare il tuo ambiente e tu puoi ricrearlo all’infinito.

---

## **Esercizio pratico (10 minuti)**

1. Lancia un container Python:

   ```bash
   docker run -it python:3.11 bash
   ```
2. Installa alcune librerie:

   ```bash
   pip install crewai qdrant-client
   ```
3. Esci e rilancia il container: scopri che le librerie **non persistono**.
   → Capirai che ogni container è **ephemeral** e **isolato**.
4. Rilancia con volume montato:

   ```bash
   docker run -it -v $(pwd)/app:/app python:3.11 bash
   ```

   Ora tutto ciò che salvi in `/app` persiste localmente.

---





# **1.2 – Differenze tra venv, Conda, VM e container**

Nel mondo dell’AI engineering esistono diversi modi per isolare un ambiente di sviluppo o di esecuzione. Prima di capire a fondo Docker, è fondamentale distinguere tra **ambienti virtuali (venv, Conda)** e **macchine virtuali (VM)**, così da comprendere dove si colloca la containerizzazione.

---

## **1. Virtualenv e venv**

Gli ambienti virtuali in Python (creati con `venv` o `virtualenv`) sono **isolatori di pacchetti**: permettono di avere versioni di librerie diverse da quelle del sistema operativo, ma non isolano il sistema vero e proprio.

```bash
python -m venv .venv
source .venv/bin/activate
pip install crewai qdrant-client
```

In questo caso:

* L’ambiente virtuale **usa sempre lo stesso Python** dell’host;
* Tutto gira **sullo stesso sistema operativo**;
* Non puoi controllare la versione di OS, CUDA o driver GPU;
* Se aggiorni globalmente una libreria come `torch`, potresti rompere un altro progetto.

È utile per **sviluppo locale**, ma non garantisce **riproducibilità perfetta**.

---

## **2. Conda environment**

Conda è uno strumento più avanzato, molto diffuso in ambito data science.
Può gestire non solo librerie Python, ma anche **pacchetti di sistema** (come `ffmpeg`, `libtorch`, `cuda`).

```bash
conda create -n crewai python=3.11
conda activate crewai
conda install pytorch cudatoolkit=12.1 -c pytorch
```

Conda risolve molti problemi di compatibilità, ma:

* Gli ambienti restano **legati al sistema operativo**;
* Non è portabile tra sistemi (un env Linux non gira su Windows);
* Le versioni dei driver e delle librerie native (CUDA, cuDNN, Rust) restano dipendenti dall’host.

In altre parole, **Conda isola le librerie, non il sistema**.

---

## **3. Virtual Machines (VM)**

Le macchine virtuali (VM) isolano tutto: sistema operativo, kernel, driver, file system.
Ogni VM è un computer completo, con un suo OS e risorse dedicate (CPU, RAM, disco).

Vantaggi:

* Isolamento totale.
* Puoi avere sistemi operativi diversi sull’host (es. Linux su Windows).

Svantaggi:

* Ogni VM pesa **diversi GB**;
* L’avvio è lento;
* Consuma molta memoria e CPU;
* Duplicare o aggiornare ambienti è costoso.

Per esempio, se un progetto CrewAI richiede 4 microservizi in VM, ognuno con un OS Linux, avresti 4 sistemi completi da mantenere, aggiornare e gestire: un incubo in produzione.

---

## **4. Docker container**

Docker rappresenta un **punto di equilibrio perfetto** tra i due mondi:

* Leggero come un ambiente virtuale;
* Isolato come una macchina virtuale.

I container non contengono un sistema operativo completo: condividono il **kernel dell’host**, ma mantengono filesystem, processi e librerie isolati.
Questo riduce drasticamente i tempi di avvio e il consumo di risorse.

### Esempio:

```bash
docker run -it python:3.11-slim bash
```

Questo comando:

* Scarica un’immagine contenente Linux minimale + Python 3.11;
* Avvia un container isolato;
* In meno di 1 secondo sei dentro un sistema pulito.

Puoi poi installare CrewAI, Qdrant, LangChain o Streamlit senza toccare il tuo sistema locale.

---

## **5. Differenze riassuntive**

| Caratteristica         | venv / virtualenv | Conda         | Virtual Machine           | Docker Container             |
| ---------------------- | ----------------- | ------------- | ------------------------- | ---------------------------- |
| Isolamento OS          | ❌                 | ❌             | ✅                         | ✅ (parziale, condiviso)      |
| Peso                   | 🔹 Leggero        | 🔹 Medio      | ⚫ Pesante                 | 🔹 Leggero                   |
| Avvio                  | Immediato         | Immediato     | Lento (minuti)            | Istantaneo                   |
| Portabilità            | ❌                 | ❌             | ✅ (con immagine)          | ✅                            |
| GPU accesso diretto    | ✅ (locale)        | ✅             | ⚠️ complesso              | ✅ (via driver NVIDIA/ROCm)   |
| Riproducibilità totale | ❌                 | ⚠️ Parziale   | ✅                         | ✅                            |
| Uso ideale             | Dev locale        | ML lab locale | Sistemi legacy o completi | Produzione AI e microservizi |

---

## **6. Esempio pratico**

Immagina di dover distribuire un sistema AI composto da:

* `CrewAI` backend per l’orchestrazione,
* `Qdrant` per la ricerca vettoriale,
* `Streamlit` come interfaccia utente.

### Con venv:

Ogni sviluppatore deve installare manualmente Python, CrewAI, Qdrant, Streamlit, configurare le porte e i database — altissimo rischio di errore.

### Con Docker:

Basta clonare il progetto e lanciare:

```bash
docker compose up -d
```

Tutto si avvia in container separati, già configurati.
L’ambiente sarà identico per ogni persona e ogni server.

---

## **7. Analogia semplice**

* **venv**: come usare scatole per tenere separati gli oggetti sulla stessa scrivania.
* **VM**: come avere scrivanie diverse in stanze diverse.
* **Docker**: come avere scatole sigillate identiche, che puoi spostare ovunque e aprire ovunque, già pronte all’uso.

---




# **1.3 – Architettura Docker: client, daemon, immagini, container e registry**

Per comprendere davvero Docker — e non solo “usarlo” — bisogna capirne la **struttura interna**: come comunica, chi fa cosa e dove avviene l’esecuzione reale dei container.
Molti sviluppatori usano Docker per anni senza conoscere la differenza tra *client* e *daemon*: questo punto serve a eliminare quella confusione, così da lavorare in modo consapevole e sicuro.

---

## **1. L’architettura generale**

Docker è composto da **quattro elementi principali**:

1. **Docker Client** → l’interfaccia con cui tu interagisci.
2. **Docker Daemon (dockerd)** → il motore che esegue davvero i container.
3. **Docker Images** → i “modelli” o blueprint dei container.
4. **Docker Containers** → le istanze in esecuzione delle immagini.
5. **Docker Registry** → il magazzino remoto dove le immagini vengono salvate e condivise.

Tutti questi componenti lavorano insieme come un sistema client-server.

---

## **2. Il Client**

Il **client** è tutto ciò che usi per comunicare con Docker:

* il comando `docker` nel terminale,
* o l’interfaccia grafica Docker Desktop.

Quando scrivi:

```bash
docker run python:3.11
```

il client **non esegue direttamente** il container.
In realtà, invia una richiesta API al Daemon Docker in background (dockerd), che è il vero motore.

Docker Client può anche collegarsi a un daemon remoto, ad esempio su un server cloud:

```bash
export DOCKER_HOST=ssh://user@server
```

In questo modo, puoi controllare container su un’altra macchina come se fossero locali.

---

## **3. Il Daemon (dockerd)**

Il **Docker Daemon** è il processo che:

* riceve comandi dal client,
* scarica immagini,
* crea, avvia e ferma container,
* gestisce volumi, reti e log.

È lui a parlare con il kernel del sistema operativo per impostare **namespaces**, **cgroups**, e filesystem copy-on-write.

In pratica, è il “cuore” del sistema Docker.

Su Linux gira come processo di sistema, su macOS e Windows è incapsulato all’interno di una VM leggera (perché Docker richiede kernel Linux).

---

### Esempio di flusso reale:

Quando esegui:

```bash
docker run -d -p 8000:8000 --name crew_backend crewai:latest
```

accade in realtà questo:

1. Il client Docker invia la richiesta al daemon (`dockerd`).
2. Il daemon controlla se l’immagine `crewai:latest` esiste localmente.

   * Se no, la scarica dal registry.
3. Crea un container a partire dall’immagine.
4. Isola il processo, assegna rete, volumi e risorse.
5. Avvia il container e monitora il suo stato.

---

## **4. Le Immagini**

Un’immagine è un **pacchetto immutabile** che contiene:

* un sistema operativo minimale (es. Debian Slim),
* tutte le dipendenze necessarie,
* e il tuo codice (CrewAI, Qdrant, Streamlit, ecc.).

Ogni immagine è composta da **layer**, ognuno dei quali rappresenta una modifica:

* layer di base (es. Python),
* layer con le librerie installate,
* layer con il codice dell’applicazione.

Esempio di Dockerfile semplificato:

```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
CMD ["python", "main.py"]
```

Quando la build parte, ogni comando crea un layer.
Docker li **mette in cache**, quindi se non modifichi `requirements.txt`, non ricostruisce tutto da zero.

---

## **5. I Container**

Il **container** è l’istanza in esecuzione di un’immagine.
È un processo isolato, con il suo filesystem, rete e risorse dedicate.

Concettualmente:

* un’immagine è come una **classe** in programmazione;
* un container è **un oggetto** istanziato da quella classe.

Puoi avere più container dalla stessa immagine:

```bash
docker run -d --name crewai1 crewai:latest
docker run -d --name crewai2 crewai:latest
```

Entrambi eseguono lo stesso codice, ma in ambienti separati.

I container sono **ephemeral**: se li elimini (`docker rm`), spariscono, ma puoi sempre ricrearli dalla stessa immagine.

---

## **6. Il Registry**

Il **registry** è un archivio remoto per immagini Docker.
I più noti sono:

* **Docker Hub** (pubblico),
* **GitHub Container Registry (GHCR)**,
* **GitLab Container Registry**,
* o registry privati aziendali (es. AWS ECR, Azure Container Registry).

Il registry è per le immagini ciò che GitHub è per il codice:
ti permette di **versionare**, **condividere** e **distribuire** in modo centralizzato.

Esempio:

```bash
docker build -t myorg/crewai-backend:1.0 .
docker push myorg/crewai-backend:1.0
```

Ora chiunque, da un’altra macchina, può eseguire:

```bash
docker run myorg/crewai-backend:1.0
```

e avrà esattamente il tuo ambiente.

---

## **7. Schema riassuntivo**

```
┌────────────────────────────┐
│        Docker Client       │
│ (CLI / Docker Desktop UI)  │
└────────────┬───────────────┘
             │ API REST
┌────────────▼───────────────┐
│       Docker Daemon        │
│     (dockerd in Linux)     │
│  ┌───────────────┬────────┐│
│  │   Images      │   Vol. ││
│  │   Containers  │   Net. ││
│  └───────────────┴────────┘│
└────────────┬────────────────
             │ Pull / Push
┌────────────▼───────────────┐
│       Docker Registry      │
│ (Docker Hub / GHCR / ECR)  │
└────────────────────────────┘
```

---

## **8. Come si lega all’AI engineering**

Capire l’architettura Docker serve anche a **diagnosticare errori frequenti** in ambienti AI:

* Se un container non parte, non è “Docker rotto”: è il daemon che non riceve o non riesegue il processo.
* Se un’immagine non viene trovata, il problema è nel registry o nei permessi di push/pull.
* Se un volume non si monta, è la gestione del layer filesystem del daemon.

In pipeline AI con CrewAI, Qdrant e Streamlit, tutto ruota intorno a questo flusso:

* il **client** (Compose o CLI) invia la configurazione;
* il **daemon** crea i servizi come container isolati;
* le **immagini** rappresentano ogni microservizio AI;
* i **registry** custodiscono le versioni deployabili.

---

## **9. Esercizio pratico (15 minuti)**

1. Esegui:

   ```bash
   docker run hello-world
   ```

   Poi verifica cosa accade nel daemon con:

   ```bash
   docker ps -a
   docker images
   docker info
   ```
2. Scarica manualmente un’immagine:

   ```bash
   docker pull python:3.11-slim
   ```

   e osserva i layer scaricati.
3. Ispeziona l’immagine:

   ```bash
   docker inspect python:3.11-slim
   ```

Capirai come **client**, **daemon** e **registry** interagiscono nel mondo reale.

---


# **1.4 – Come Docker gestisce librerie pesanti (Torch, CUDA, ROCm, Transformers)**

Quando si containerizza un’applicazione AI, non si parla più solo di Python e dipendenze leggere.
L’ambiente deve spesso includere:

* **PyTorch** o **TensorFlow**, librerie enormi con binding nativi in C++ o CUDA;
* **CUDA Toolkit** o **ROCm**, per l’accelerazione su GPU NVIDIA o AMD;
* **Transformers** e modelli LLM, spesso di diversi GB di peso.

Docker può gestire tutto questo in modo efficiente, ma serve capire come funziona “sotto il cofano”.

---

## **1. Docker e le librerie native**

Una libreria come `torch` non è solo Python puro: include componenti compilati in C/C++ e spesso richiede l’accesso diretto a driver e device hardware.

Nel sistema host, queste librerie vengono installate con il supporto nativo del sistema operativo (es. `/usr/lib/cuda`).
Nel container, invece, **non esiste nulla di preinstallato**: devi fornire tu l’ambiente completo.

Docker lo risolve in due modi:

1. usando **immagini base specifiche per GPU**, già predisposte da NVIDIA o AMD;
2. esponendo i **driver GPU dell’host** al container tramite un layer d’integrazione (NVIDIA Container Toolkit o ROCm runtime).

---

## **2. Immagini base NVIDIA e CUDA**

NVIDIA mantiene immagini ufficiali con CUDA e PyTorch preconfigurati, ad esempio:

* `nvidia/cuda`
* `pytorch/pytorch`
* `tensorflow/tensorflow`

Queste immagini contengono:

* il sistema operativo base (Ubuntu o Debian),
* CUDA Toolkit (compilatori, librerie, runtime),
* i driver utente per l’accelerazione GPU,
* e a volte anche PyTorch preinstallato.

Esempio:

```dockerfile
FROM pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime

WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
CMD ["python", "main.py"]
```

Quando esegui il container, puoi dare accesso alla GPU dell’host:

```bash
docker run --gpus all my-ai-container
```

Il container non ha bisogno di driver NVIDIA interni: utilizza quelli **già presenti sull’host**, esposti attraverso il toolkit.

---

## **3. NVIDIA Container Toolkit**

Il **NVIDIA Container Toolkit** è ciò che permette a Docker di parlare con la GPU.
Funziona come un ponte tra il sistema host e il container.

Installazione (Linux):

```bash
sudo apt install -y nvidia-container-toolkit
sudo systemctl restart docker
```

Dopo questa configurazione, puoi:

* lanciare container con GPU accessibile (`--gpus all`);
* monitorare la GPU dal container (`nvidia-smi`);
* eseguire modelli Torch e TensorFlow accelerati.

Senza questo toolkit, Docker non può accedere alle GPU dell’host, perché il kernel non “vede” i device.

---

## **4. ROCm e GPU AMD**

Per GPU AMD, il meccanismo è simile ma basato su **ROCm** (Radeon Open Compute).
AMD fornisce immagini ufficiali con supporto ROCm preinstallato, ad esempio:

* `rocm/pytorch`
* `rocm/tensorflow`

Esempio:

```dockerfile
FROM rocm/pytorch:latest
WORKDIR /app
COPY . .
CMD ["python", "train.py"]
```

L’host deve avere i driver ROCm installati e accessibili.
Docker non gestisce direttamente la GPU AMD: la espone come device `/dev/kfd` o `/dev/dri`, e il container la utilizza tramite i runtime ROCm interni.

Avvio:

```bash
docker run --device=/dev/kfd --device=/dev/dri my-rocm-container
```

---

## **5. Transformers e modelli di grandi dimensioni**

Le librerie **Transformers** e **Diffusers** di Hugging Face portano un’altra sfida: i modelli sono enormi (GB di pesi binari), spesso scaricati da remoto.

Best practice:

* monta una **cache condivisa** per i modelli, per non riscaricarli a ogni build;
* non includere i pesi dentro l’immagine Docker;
* usa un volume dedicato o un percorso esterno.

Esempio in Compose:

```yaml
services:
  crewai:
    build: .
    volumes:
      - ./models:/root/.cache/huggingface
```

In questo modo:

* i modelli restano persistenti anche se ricrei il container;
* più container possono condividere la stessa cache.

---

## **6. Dimensioni delle immagini e ottimizzazione**

Quando si aggiungono PyTorch, CUDA e Transformers, le immagini possono facilmente superare i **5–10 GB**.
Per ottimizzarle:

1. **Usa multi-stage build**

   * Compila o installa solo ciò che serve in un primo stage.
   * Copia solo i binari finali nello stage runtime.

   ```dockerfile
   FROM pytorch/pytorch:2.3.0-cuda12.1-cudnn8-devel as build
   RUN pip install crewai qdrant-client

   FROM pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime
   COPY --from=build /opt/conda /opt/conda
   CMD ["python", "main.py"]
   ```

2. **Evita apt inutili**

   * Ogni `RUN apt-get install` crea un layer.
   * Pulisci la cache (`rm -rf /var/lib/apt/lists/*`).

3. **Usa immagini “slim” o “runtime”**

   * Le immagini `-devel` contengono compilatori e header (necessari solo in build).
   * In produzione basta `-runtime`.

---

## **7. Testare GPU e librerie in container**

Per verificare che Docker stia usando correttamente la GPU:

```bash
docker run --gpus all --rm pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime nvidia-smi
```

Output atteso:

```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 550.40       Driver Version: 550.40       CUDA Version: 12.1     |
| GPU Name: RTX 4090      Memory Usage: 2345MiB / 24576MiB                    |
+-----------------------------------------------------------------------------+
```

Poi, per testare Torch:

```bash
docker run --gpus all -it pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime python -c "import torch; print(torch.cuda.is_available())"
```

→ `True`

Questo conferma che il container accede ai driver GPU host e può eseguire modelli accelerati.

---

## **8. Integrazione con CrewAI e Qdrant**

Quando costruisci pipeline reali con **CrewAI**, **Qdrant** e **LLM**, separa sempre i componenti:

* il container “AI compute” (PyTorch, Transformers, CUDA/ROCm);
* il container “vector DB” (Qdrant o Milvus);
* e il container “interface” (Streamlit o FastAPI).

Questo approccio:

* evita conflitti tra librerie native (Torch, Rust, SQLite, ecc.),
* riduce le dimensioni di ciascun container,
* permette di scalare indipendentemente il componente AI compute.

Esempio di Compose parziale:

```yaml
services:
  ai-compute:
    build: ./ai
    deploy:
      resources:
        reservations:
          devices:
            - capabilities: [gpu]
  qdrant:
    image: qdrant/qdrant:v1.10.0
    volumes:
      - qdrant_data:/qdrant/storage
  streamlit:
    build: ./ui
    ports:
      - "8501:8501"
volumes:
  qdrant_data:
```

---

## **9. Riepilogo concettuale**

| Elemento                                    | Ruolo in Docker                                          | Note                                                     |
| ------------------------------------------- | -------------------------------------------------------- | -------------------------------------------------------- |
| **PyTorch / TensorFlow**                    | Librerie Python con binding C/CUDA                       | Devono essere installate su immagini compatibili con GPU |
| **CUDA / ROCm**                             | Layer di accesso GPU                                     | Esposto dal sistema host, non incluso nei container      |
| **Transformers / Diffusers**                | Framework LLM e modelli                                  | Gestire cache e volumi condivisi                         |
| **NVIDIA Container Toolkit / ROCm runtime** | Ponte hardware → container                               | Necessario per esporre device GPU                        |
| **Best practice**                           | Usa immagini preconfigurate, multi-stage e cache modelli | Evita build pesanti e duplicazione di dati               |

---




# **1.5 – Setup ambiente (Windows): installazione Docker Desktop e configurazione GPU (NVIDIA o ROCm)**

Su Windows, Docker non viene eseguito nativamente: lavora all’interno di una macchina virtuale Linux gestita dal **motore WSL 2 (Windows Subsystem for Linux)**.
Questo è il componente che permette a Docker di avere un vero kernel Linux e quindi di eseguire correttamente container basati su immagini come `python:3.11-slim`, `pytorch/pytorch`, o `qdrant/qdrant`.

L’obiettivo di questo setup è garantire:

* un’installazione pulita e stabile di Docker Desktop;
* l’attivazione di **WSL 2** e del **kernel Linux**;
* il corretto **accesso alla GPU** (NVIDIA o AMD) da parte dei container;
* la possibilità di eseguire stack AI completi (CrewAI + Qdrant + Streamlit) con accelerazione hardware.

---

## **1. Prerequisiti di sistema**

### Requisiti minimi

* **Windows 10** (versione 2004 o superiore) oppure **Windows 11**
* **64 bit**
* **CPU con virtualizzazione hardware abilitata** (Intel VT-x o AMD-V)
* Almeno **8 GB di RAM** (consigliati 16 GB per AI)
* Connessione Internet per scaricare immagini e tool

### Requisiti GPU

* **Per NVIDIA:** driver 470+ e toolkit CUDA 11 o superiore
* **Per AMD:** driver ROCm 6.0+ con supporto a HIP (solo Windows 11 attualmente sperimentale)

---

## **2. Installazione di WSL 2**

WSL 2 è il motore Linux su cui Docker Desktop si appoggia.
Per installarlo:

Apri **PowerShell come Amministratore** e digita:

```bash
wsl --install
```

Questo comando:

* attiva i componenti “Piattaforma macchina virtuale” e “Sottosistema Windows per Linux”;
* installa automaticamente Ubuntu come distribuzione predefinita;
* abilita il kernel Linux.

Dopo il riavvio, verifica che tutto sia attivo:

```bash
wsl -l -v
```

Dovresti vedere un output simile a:

```
  NAME      STATE           VERSION
* Ubuntu    Running         2
```

Se `VERSION` è 1, aggiorna con:

```bash
wsl --set-version Ubuntu 2
```

---

## **3. Installazione di Docker Desktop**

1. Vai al sito ufficiale:
   🔗 [https://www.docker.com/products/docker-desktop/](https://www.docker.com/products/docker-desktop/)
2. Scarica **Docker Desktop for Windows**.
3. Durante l’installazione, **assicurati di selezionare “Use WSL 2 based engine”**.
4. Dopo l’installazione, riavvia il sistema.
5. Avvia Docker Desktop e apri il terminale PowerShell o Ubuntu WSL per testare:

   ```bash
   docker run hello-world
   ```

   Se il messaggio dice “Hello from Docker!”, l’installazione è riuscita.

---

## **4. Configurazione GPU – NVIDIA**

Per usare PyTorch o TensorFlow con accelerazione GPU all’interno di container, serve il **NVIDIA Container Toolkit**.

### Passaggi

1. Assicurati che i driver NVIDIA siano aggiornati:

   * Apri `nvidia-smi` nel prompt.
     Se restituisce un output valido, i driver sono attivi.

2. Installa **NVIDIA Container Toolkit** (dalla WSL Ubuntu):

   ```bash
   distribution=$(. /etc/os-release;echo $ID$VERSION_ID)
   curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg
   curl -s -L https://nvidia.github.io/libnvidia-container/$distribution/libnvidia-container.list | \
     sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list
   sudo apt-get update
   sudo apt-get install -y nvidia-container-toolkit
   sudo systemctl restart docker
   ```

3. Verifica:

   ```bash
   docker run --rm --gpus all nvidia/cuda:12.1-base nvidia-smi
   ```

   Se compare la tua GPU, Docker è configurato per CUDA.

4. Ora puoi eseguire modelli CrewAI o Torch nel container:

   ```bash
   docker run --gpus all -it pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime python -c "import torch; print(torch.cuda.is_available())"
   ```

   Output atteso: `True`

---

## **5. Configurazione GPU – AMD (ROCm)**

> ⚠️ Su Windows il supporto ROCm via Docker è ancora **sperimentale**.
> Si consiglia di usare **Ubuntu WSL** con ROCm installato o, meglio, una macchina Linux nativa.

Per sistemi che supportano ROCm (es. RX 7900 XTX, MI 210, ecc.):

1. Installa driver ROCm per Windows 11:
   🔗 [https://www.amd.com/en/developer/resources/rocm.html](https://www.amd.com/en/developer/resources/rocm.html)
2. Installa Docker Desktop come sopra.
3. Apri WSL Ubuntu e verifica la presenza dei device:

   ```bash
   ls /dev | grep kfd
   ```
4. Avvia container ROCm:

   ```bash
   docker run --device=/dev/kfd --device=/dev/dri rocm/pytorch:latest
   ```

---

## **6. Test finale**

Una volta completata l’installazione:

```bash
docker run hello-world
docker run -it python:3.11-slim bash
```

Se entrambi i comandi funzionano, Docker e WSL 2 sono configurati correttamente.

Per GPU:

* Verifica CUDA: `docker run --gpus all nvidia/cuda:12.1-base nvidia-smi`
* Oppure PyTorch: `docker run --gpus all pytorch/pytorch python -c "import torch; print(torch.cuda.is_available())"`

---

## **7. Consigli pratici per progetti AI**

* **Disattiva “Use Windows containers”**: assicurati che Docker Desktop usi container **Linux-based**.
* **Assegna risorse adeguate** (Settings → Resources):
  CPU ≥ 4 core, RAM ≥ 8 GB, GPU enabled.
* **Condividi le directory di lavoro** (Settings → Resources → File Sharing): aggiungi la cartella del progetto.
* **Evita di costruire immagini dentro WSL** se usi IDE Windows: costruiscile da terminale Docker Desktop o da VS Code + Docker Extension per mantenere il contesto coerente.
* **Aggiorna WSL regolarmente**:

  ```bash
  wsl --update
  ```

---


# **1.6 – Comandi base e ciclo di vita di un container**

Quando esegui un container, Docker lo tratta come un **processo isolato**: può essere avviato, messo in pausa, riavviato o rimosso.
Per gestirlo, Docker fornisce una serie di comandi CLI che ti permettono di **controllare ogni fase del suo ciclo di vita**.

---

## **1. Concetto di ciclo di vita**

Un container nasce da un’immagine ed esegue un processo principale (il *command* o *entrypoint* definito nel Dockerfile).
Dalla creazione alla rimozione, può passare attraverso vari stati:

```
created → running → stopped → removed
```

Ogni fase è gestita da comandi specifici, che vediamo subito.

---

## **2. `docker ps` – Visualizzare i container attivi**

Mostra la lista dei container attualmente in esecuzione.

```bash
docker ps
```

Esempio di output:

```
CONTAINER ID   IMAGE                  COMMAND                  STATUS         PORTS                  NAMES
f2a45b8c11df   qdrant/qdrant:latest   "/usr/bin/qdrant"        Up 5 minutes   6333/tcp, 6334/tcp    qdrant_db
d3a21ce6c442   crewai:latest          "python main.py"         Up 2 minutes   8080/tcp              crew_backend
```

### Opzioni utili:

* `docker ps -a` → mostra **tutti i container**, anche quelli stoppati.
* `docker ps -q` → mostra solo gli ID (utile negli script).
* `docker ps --filter "status=exited"` → filtra per stato.

 *Uso tipico in AI pipelines:*
Verificare che `qdrant`, `crewai` e `streamlit` siano effettivamente attivi in uno stack multi-container.

---

## **3. `docker exec` – Entrare o eseguire comandi dentro un container**

Permette di eseguire un comando in un container già in esecuzione, o di aprire una shell interattiva.

```bash
docker exec -it <container_name> bash
```

Esempio:

```bash
docker exec -it crew_backend bash
```

Ora sei “dentro” il container come se fosse un piccolo Linux isolato.
Puoi navigare, leggere log, o testare Python:

```bash
python
import crewai
```

### Varianti:

* `docker exec crew_backend ls /app` → esegue un singolo comando e restituisce l’output.
* `-i` = interattivo (input), `-t` = terminale (TTY).

 *Esempio pratico:*
Se il container `crewai` non risponde, puoi entrare e controllare il file `/app/main.py` o la presenza delle chiavi `.env`.

---

## **4. `docker logs` – Leggere i log del container**

Mostra l’output standard (`stdout` e `stderr`) del processo principale del container.
È uno dei comandi più usati per il debugging.

```bash
docker logs crew_backend
```

Esempio di output:

```
[INFO] CrewAI server started on port 8080
[INFO] Connected to Qdrant at qdrant_db:6333
[WARNING] Missing OpenAI API key - using fallback model
```

### Opzioni utili:

* `docker logs -f crew_backend` → *follow mode*, segue in tempo reale i log (come `tail -f`).
* `docker logs --since 10m crew_backend` → mostra solo gli ultimi 10 minuti.
* `docker logs -n 50 crew_backend` → ultimi 50 log lines.

 *Uso tipico:*
Quando un microservizio AI fallisce all’avvio, questo comando mostra errori di import, di connessione o di chiavi API mancanti.

---

## **5. `docker inspect` – Analizzare in profondità container o immagini**

Fornisce tutte le informazioni tecniche di un container o di un’immagine in formato JSON.
È fondamentale per capire configurazioni, reti, volumi e variabili d’ambiente.

```bash
docker inspect crew_backend
```

Output (estratto semplificato):

```json
[
  {
    "Id": "d3a21ce6c442...",
    "State": { "Status": "running", "Pid": 1342 },
    "Mounts": [
      { "Source": "/home/michael/app", "Destination": "/app" }
    ],
    "NetworkSettings": {
      "IPAddress": "172.18.0.3",
      "Ports": { "8080/tcp": [{ "HostPort": "8080" }] }
    },
    "Config": {
      "Env": ["OPENAI_API_KEY=sk-...", "MODE=production"],
      "Cmd": ["python", "main.py"]
    }
  }
]
```

### Opzioni utili:

* `docker inspect --format='{{.NetworkSettings.IPAddress}}' crew_backend`
  → mostra solo l’IP interno.
* `docker inspect -f '{{.Config.Env}}' crew_backend`
  → mostra le variabili d’ambiente.

 *Uso tipico:*
Scoprire su quale rete interna è connesso un container CrewAI per permettere a Streamlit o Qdrant di comunicare correttamente.

---

## **6. Comandi complementari del ciclo di vita**

### Avvio e stop container

```bash
docker start crew_backend
docker stop crew_backend
```

### Riavvio rapido

```bash
docker restart crew_backend
```

### Creazione e rimozione

```bash
docker run -d --name crew_backend crewai:latest
docker rm crew_backend
```

 *Nota:* un container rimosso non cancella l’immagine da cui è stato creato.

---

## **7. Visualizzare immagini e volumi**

Per vedere le immagini salvate localmente:

```bash
docker images
```

Per vedere i volumi (dove risiedono i dati persistenti):

```bash
docker volume ls
```

Esempio tipico in uno stack CrewAI:

```
REPOSITORY          TAG       SIZE
crewai              latest    2.3GB
qdrant/qdrant       v1.10.0   650MB
python              3.11-slim 140MB
```

---

## **8. Esercizio pratico (20 minuti)**

1. Avvia un container Python:

   ```bash
   docker run -d --name pytest python:3.11-slim sleep 300
   ```
2. Verifica che sia attivo:

   ```bash
   docker ps
   ```
3. Entra nel container:

   ```bash
   docker exec -it pytest bash
   ```
4. Installa qualcosa al suo interno (es. `pip install numpy`), poi esci.
5. Leggi i log:

   ```bash
   docker logs pytest
   ```
6. Ispeziona:

   ```bash
   docker inspect pytest
   ```
7. Ferma e rimuovi:

   ```bash
   docker stop pytest
   docker rm pytest
   ```

Questo ciclo ti mostra come **nascita, esecuzione e distruzione** di un container avvengono in modo trasparente e controllato.

---

## **9. Riepilogo pratico**

| Comando                | Funzione principale                  | Uso tipico               |
| ---------------------- | ------------------------------------ | ------------------------ |
| `docker ps`            | Elenca container attivi              | Controllo dello stato    |
| `docker exec`          | Entra o esegue comandi nel container | Debug o test rapido      |
| `docker logs`          | Legge i log del container            | Diagnosi di errori       |
| `docker inspect`       | Mostra configurazione completa       | Analisi rete, env, mount |
| `docker start/stop/rm` | Gestione ciclo di vita               | Riavvio, cleanup         |

---



# **1.7 – Gestione ambienti AI interattivi (`docker run -it python:3.11 bash`)**

Molti sviluppatori AI lavorano con ambienti virtuali o notebook locali, ma questo spesso porta a conflitti tra versioni di librerie, Python o CUDA.
Con Docker puoi creare **ambienti di sviluppo temporanei e isolati**, pronti all’uso, perfetti per testare rapidamente nuove librerie, modelli o configurazioni di CrewAI.

---

## **1. Concetto di ambiente interattivo**

Un container Docker può essere usato in due modi:

* come **servizio**, in esecuzione continua (es. un backend CrewAI, un database Qdrant);
* oppure come **ambiente interattivo**, dove lavori dentro una shell Linux pulita, come se fosse una macchina virtuale leggera.

Il comando chiave è:

```bash
docker run -it python:3.11 bash
```

Vediamo cosa succede in dettaglio.

---

## **2. Analisi del comando**

| Parte         | Significato                                                        |
| ------------- | ------------------------------------------------------------------ |
| `docker run`  | crea e avvia un container basato su un’immagine                    |
| `-i`          | abilita l’input interattivo (stdin aperto)                         |
| `-t`          | assegna un terminale TTY per interazione umana                     |
| `python:3.11` | immagine base ufficiale di Python (Debian + Python 3.11)           |
| `bash`        | comando da eseguire all’avvio del container (avvia una shell Bash) |

Il risultato è un prompt interattivo dentro un sistema Linux minimale, con Python già installato.

Esempio:

```
root@0a3b82f3d87c:/# python
Python 3.11.9 (main, May 10 2024, 10:11:00)
>>> import sys
>>> sys.version
'3.11.9'
```

Ora sei **dentro un container** isolato dal tuo sistema Windows o macOS, e tutto ciò che fai (installazioni, file, modifiche) resta confinato lì dentro.

---

## **3. Caratteristiche di un ambiente interattivo Docker**

1. **Pulito:** ogni container parte sempre dallo stesso stato iniziale.
2. **Isolato:** non interferisce con librerie o file dell’host.
3. **Temporaneo:** se lo chiudi senza salvare, sparisce (utile per test rapidi).
4. **Reproducibile:** puoi ricrearlo identico in ogni momento.
5. **Leggero:** avvio in meno di un secondo.

 È perfetto per testare rapidamente:

* librerie nuove (`pip install crewai qdrant-client`);
* modelli (`from transformers import pipeline`);
* bug di compatibilità (`import torch; torch.cuda.is_available()`).

---

## **4. Lavorare dentro il container**

Una volta dentro la shell:

```bash
root@0a3b82f3d87c:/#
```

Puoi usare comandi Linux e Python normalmente:

```bash
apt update && apt install nano -y
pip install crewai qdrant-client langchain
python
```

Tutto funzionerà come in un vero sistema Linux, ma:

* i file restano **solo nel container**;
* quando esci (`exit` o `Ctrl+D`), il container si ferma;
* al riavvio sarà “vuoto” se non hai salvato nulla su un volume (vedremo tra poco come farlo).

---

## **5. Uscire e rientrare**

Esci con:

```bash
exit
```

Il container resta fermo ma non scompare.
Puoi vederlo con:

```bash
docker ps -a
```

E rientrare con:

```bash
docker start -ai <container_id>
```

oppure eliminarlo:

```bash
docker rm <container_id>
```

 *Consiglio*: dai un nome ai tuoi container per non confonderli:

```bash
docker run -it --name crew_env python:3.11 bash
```

Poi puoi riprenderlo facilmente:

```bash
docker start -ai crew_env
```

---

## **6. Montare una cartella locale (persistenza)**

Per evitare che tutto vada perso alla chiusura, puoi **montare una directory dell’host** nel container:

```bash
docker run -it -v %cd%:/app python:3.11 bash   # su Windows PowerShell
```

oppure

```bash
docker run -it -v $(pwd):/app python:3.11 bash # su Linux/macOS
```

Ora la cartella corrente del tuo computer è visibile in `/app` dentro il container.
Puoi crearci file, script o notebook:

```bash
cd /app
nano test.py
python test.py
```

Tutto resta salvato anche dopo l’uscita, perché i file vivono nel filesystem dell’host.

---

## **7. Eseguire Python direttamente**

Puoi saltare la shell e avviare Python in un solo step:

```bash
docker run -it python:3.11
```

oppure eseguire un comando singolo:

```bash
docker run --rm python:3.11 python -c "print('Hello AI World')"
```

L’opzione `--rm` elimina automaticamente il container una volta terminato.

 *Utile per test rapidi di librerie CrewAI, LangChain, HuggingFace o Qdrant-client senza installarle localmente.*

---

## **8. Testare la GPU in un ambiente interattivo**

Se hai configurato NVIDIA Container Toolkit:

```bash
docker run -it --gpus all pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime bash
```

Poi dentro:

```bash
python -c "import torch; print(torch.cuda.is_available())"
```

→ `True` significa che Docker sta usando la tua GPU correttamente.

---

## **9. Pulizia e gestione**

Dopo aver terminato gli esperimenti, puoi pulire facilmente:

```bash
docker ps -a            # mostra tutti i container
docker rm <id>          # rimuove un container
docker images           # mostra immagini
docker rmi python:3.11  # rimuove un’immagine se vuoi liberare spazio
```

 *Docker Desktop mostra tutto anche graficamente, utile per i primi tempi.*

---


## **11. In sintesi**

| Azione                       | Comando                           |
| ---------------------------- | --------------------------------- |
| Avvia ambiente interattivo   | `docker run -it python:3.11 bash` |
| Assegna un nome al container | `--name crew_env`                 |
| Monta una directory locale   | `-v %cd%:/app`                    |
| Riaccedi al container        | `docker start -ai crew_env`       |
| Elimina container e immagine | `docker rm` / `docker rmi`        |

---





#  **Docker – Recap dei Comandi Fondamentali**

| **Comando**                                                      | **Funzione**                                              | **Esempio pratico / Descrizione**                                       |
| ---------------------------------------------------------------- | --------------------------------------------------------- | ----------------------------------------------------------------------- |
| `docker run <img>`                                               | Crea e avvia un container da un’immagine                  | `docker run hello-world` → esegue un test di installazione              |
| `docker run -it <img> bash`                                      | Avvia un container **interattivo** con terminale          | `docker run -it python:3.11 bash` → entra in un ambiente Python isolato |
| `docker run -it --name <nome>`                                   | Crea container con nome personalizzato                    | `docker run -it --name crew_env python:3.11 bash`                       |
| `docker run --rm <img>`                                          | Esegue container temporaneo e lo elimina alla chiusura    | `docker run --rm python:3.11 python -c "print('Hello')"`                |
| `docker run -v <path_host>:<path_container>`                     | Monta una cartella locale dentro il container             | `docker run -it -v %cd%:/app python:3.11 bash`                          |
| `docker run --gpus all <img>`                                    | Espone GPU NVIDIA/AMD al container                        | `docker run --gpus all pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime`   |
| `docker ps`                                                      | Mostra i container attivi                                 | `docker ps` → elenca solo quelli in esecuzione                          |
| `docker ps -a`                                                   | Mostra tutti i container, inclusi quelli fermati          | Utile per controllare container creati o terminati                      |
| `docker exec -it <container> bash`                               | Apre shell Bash dentro un container attivo                | `docker exec -it crew_backend bash`                                     |
| `docker exec <container> <cmd>`                                  | Esegue un singolo comando dentro un container             | `docker exec crew_backend ls /app`                                      |
| `docker logs <container>`                                        | Mostra i log standard del container                       | `docker logs -f crew_backend` → segue in tempo reale i log              |
| `docker inspect <container>`                                     | Mostra info dettagliate (rete, mount, env, porte)         | `docker inspect qdrant_db`                                              |
| `docker inspect -f '{{.NetworkSettings.IPAddress}}' <container>` | Estrae solo l’IP interno del container                    | utile per connessioni manuali tra servizi                               |
| `docker images`                                                  | Elenca tutte le immagini salvate localmente               | Mostra repository, tag, dimensione e ID                                 |
| `docker pull <img>`                                              | Scarica un’immagine dal registry (Docker Hub, GHCR, ecc.) | `docker pull python:3.11-slim`                                          |
| `docker build -t <nome>:<tag> .`                                 | Crea un’immagine a partire da un Dockerfile               | `docker build -t crewai-backend:1.0 .`                                  |
| `docker tag <img> <registry>/<repo>:<tag>`                       | Aggiunge un tag per pushare un’immagine                   | `docker tag crewai:1.0 myorg/crewai:1.0`                                |
| `docker push <repo>`                                             | Carica un’immagine sul registry remoto                    | `docker push myorg/crewai:1.0`                                          |
| `docker start <container>`                                       | Riavvia un container fermato                              | `docker start crew_backend`                                             |
| `docker stop <container>`                                        | Ferma un container attivo                                 | `docker stop crew_backend`                                              |
| `docker restart <container>`                                     | Riavvia il container (stop + start)                       | utile per applicare aggiornamenti                                       |
| `docker rm <container>`                                          | Rimuove un container fermo                                | `docker rm crew_backend`                                                |
| `docker rmi <image>`                                             | Rimuove un’immagine locale                                | `docker rmi python:3.11-slim`                                           |
| `docker volume ls`                                               | Elenca i volumi persistenti                               | utile per dataset, modelli o DB                                         |
| `docker network ls`                                              | Elenca le reti Docker                                     | utile per capire come comunicano i microservizi                         |
| `docker compose up -d`                                           | Avvia più container definiti in `docker-compose.yml`      | `docker compose up -d` → lancia CrewAI + Qdrant + Streamlit             |
| `docker compose down`                                            | Ferma e rimuove tutti i container dello stack             | Cleanup completo di uno stack AI                                        |

---




#  **Esercizi sui comandi base Docker**

---

##  **SEZIONE 1 – TRACCE (per esercitarsi in autonomia)**

### 🔹 **Esercizio 1 – Primo container interattivo**

Crea un container interattivo basato su `python:3.11`, entra al suo interno, installa qualche libreria AI (es. `crewai` e `qdrant-client`), e verifica che funzioni.
Chiudi il container e scopri se al riavvio le librerie sono ancora presenti.

---

### 🔹 **Esercizio 2 – Persistenza tramite volume**

Ripeti l’esercizio 1, ma questa volta monta una cartella locale in `/app` dentro il container, salva un file Python e verifica che resti sul disco anche dopo aver chiuso il container.

---

### 🔹 **Esercizio 3 – Creare e rinominare container**

Crea due container basati su `python:3.11`:

* uno con il nome `crew_test`
* uno con il nome `qdrant_test`

Elenca i container, fermali e rimuovili entrambi.

---

### 🔹 **Esercizio 4 – Esaminare container attivi**

Esegui un container in background (`-d`) basato su `python:3.11-slim` che esegua il comando:

```bash
python -c "import time; [print('running...') or time.sleep(2) for _ in range(5)]"
```

Poi:

1. Controlla che sia attivo.
2. Leggi i log del container.
3. Ispeziona la sua configurazione (IP, comando, stato).
4. Attendi che termini e verifica lo stato con `docker ps -a`.

---

### 🔹 **Esercizio 5 – Interagire con un container esistente**

Crea un container in background con:

```bash
docker run -d --name crew_env python:3.11 sleep 300
```

Poi:

* entra dentro il container con `docker exec -it`;
* crea un file `test.txt`;
* leggi il contenuto dall’host senza entrare nel container;
* infine fermalo e rimuovilo.

---

### 🔹 **Esercizio 6 – Analisi dettagliata**

Crea un container basato su `python:3.11`, assegnagli un nome (`analyze_me`) e avvialo.
Usa `docker inspect` per:

1. Estrarre il suo indirizzo IP interno;
2. Verificare il comando di avvio;
3. Controllare se ha variabili d’ambiente predefinite.

---

### 🔹 **Esercizio 7 – Pulizia e gestione immagini**

1. Elenca le immagini presenti sul tuo sistema.
2. Cancella le immagini che non ti servono.
3. Rimuovi tutti i container non più in uso.
4. Pulisci il sistema con `docker system prune`.
5. Controlla quanto spazio hai liberato.


---

#  **SEZIONE 2 – SOLUZIONI GUIDATE PASSO PASSO**

---

##  **Esercizio 1 – Primo container interattivo**

**1. Avvia il container:**

```bash
docker run -it python:3.11 bash
```

**2. All’interno:**

```bash
pip install crewai qdrant-client
python -c "import crewai, qdrant_client; print('ok')"
```

**3. Esci:**

```bash
exit
```

**4. Controlla lo stato:**

```bash
docker ps -a
```

Il container è fermo ma ancora presente.
Riavvialo:

```bash
docker start -ai <container_id>
```

Verifica: le librerie **non sono più installate** — il container è effimero.

---

##  **Esercizio 2 – Persistenza tramite volume**

**1. Avvia il container con volume montato:**

```bash
docker run -it -v %cd%:/app python:3.11 bash
```

(su Linux/macOS: `-v $(pwd):/app`)

**2. All’interno del container:**

```bash
cd /app
echo "print('Hello from Docker')" > hello.py
python hello.py
```

**3. Esci e controlla nella cartella locale:**
Il file `hello.py` è rimasto → la persistenza funziona.

---

##  **Esercizio 3 – Creare e rinominare container**

**1. Crea i container:**

```bash
docker run -it --name crew_test python:3.11 bash
docker run -it --name qdrant_test python:3.11 bash
```

(esci subito da entrambi con `exit`)

**2. Elenca i container:**

```bash
docker ps -a
```

**3. Ferma e rimuovi:**

```bash
docker stop crew_test qdrant_test
docker rm crew_test qdrant_test
```

---

##  **Esercizio 4 – Esaminare container attivi**

**1. Esegui in background:**

```bash
docker run -d --name test_logger python:3.11-slim python -c "import time; [print('running...') or time.sleep(2) for _ in range(5)]"
```

**2. Controlla stato:**

```bash
docker ps
```

**3. Leggi log:**

```bash
docker logs test_logger
```

**4. Ispeziona:**

```bash
docker inspect test_logger
```

**5. Dopo 10 secondi:**

```bash
docker ps -a
```

Il container avrà `STATUS: Exited`.

---

##  **Esercizio 5 – Interagire con un container esistente**

**1. Avvia container in background:**

```bash
docker run -d --name crew_env python:3.11 sleep 300
```

**2. Entra dentro:**

```bash
docker exec -it crew_env bash
```

**3. Crea file:**

```bash
echo "Docker test OK" > /tmp/test.txt
exit
```

**4. Leggi contenuto da host:**

```bash
docker exec crew_env cat /tmp/test.txt
```

**5. Ferma e rimuovi:**

```bash
docker stop crew_env
docker rm crew_env
```

---

##  **Esercizio 6 – Analisi dettagliata**

**1. Crea container:**

```bash
docker run -d --name analyze_me python:3.11 sleep 60
```

**2. Estrarre IP:**

```bash
docker inspect -f '{{.NetworkSettings.IPAddress}}' analyze_me
```

**3. Comando di avvio:**

```bash
docker inspect -f '{{.Config.Cmd}}' analyze_me
```

**4. Variabili d’ambiente:**

```bash
docker inspect -f '{{.Config.Env}}' analyze_me
```

---

##  **Esercizio 7 – Pulizia e gestione immagini**

**1. Elenca immagini e container:**

```bash
docker images
docker ps -a
```

**2. Cancella tutto ciò che non serve:**

```bash
docker rm $(docker ps -aq)
docker rmi $(docker images -q)
```

**3. Pulizia generale:**

```bash
docker system prune -af
```

**4. Controlla spazio:**

```bash
docker system df
```

---








# **1.8 – Persistenza notebook/data (`-v`, `--mount`, `--network`)**

---

## **1. Il problema della volatilità nei container**

Per impostazione predefinita, i container Docker sono **temporanei**.
Se li elimini, tutto ciò che era stato scritto dentro (file, notebook, database, modelli scaricati) sparisce.
Questo comportamento è ottimo per test e build pulite, ma in progetti AI è **inaccettabile**:

* vogliamo mantenere **dataset e notebook** tra sessioni,
* salvare **cache di modelli Hugging Face**,
* preservare **indici vettoriali di Qdrant** o **file SQLite di CrewAI**.

Docker offre due modi principali per salvare i dati:

* **Volumi** (gestiti da Docker, con `-v` o `--mount`)
* **Bind mount** (collega una cartella dell’host)

Vediamoli nel dettaglio.

---

## **2. Persistenza con `-v` (bind mount)**

Il metodo più semplice e immediato per rendere persistenti i dati è collegare una **cartella locale dell’host** a una directory del container.

### Esempio:

```bash
docker run -it -v %cd%:/app python:3.11 bash   # Windows PowerShell
```

oppure su Linux/macOS:

```bash
docker run -it -v $(pwd):/app python:3.11 bash
```

All’interno del container, tutto ciò che scrivi in `/app` viene salvato nella tua directory locale.

### Test rapido:

```bash
cd /app
echo "print('Persistente!')" > test.py
exit
```

Il file `test.py` esiste ancora nel tuo computer locale.

 **Uso tipico in AI:**

* salvare notebook Jupyter (`.ipynb`);
* scrivere output JSON o CSV di CrewAI;
* mantenere la cache Hugging Face (`/root/.cache/huggingface`).

---

## **3. Persistenza con `--mount` (volumi gestiti da Docker)**

`--mount` è un metodo più moderno e leggibile rispetto a `-v`.
Offre maggiore controllo e chiarezza, soprattutto negli script o nei file Compose.

### Sintassi:

```bash
docker run -it --mount type=bind,source=%cd%,target=/app python:3.11 bash
```

Equivalente a `-v %cd%:/app`, ma più esplicito e robusto.
Puoi anche montare **volumi gestiti da Docker**, non solo cartelle locali.

### Esempio con volume Docker:

```bash
docker volume create crew_data
docker run -it --mount source=crew_data,target=/app python:3.11 bash
```

Tutto ciò che salvi in `/app` è conservato nel volume, anche se elimini il container.
Puoi elencare i volumi:

```bash
docker volume ls
```

e ispezionarli:

```bash
docker volume inspect crew_data
```

 **Uso tipico in AI:**

* salvare database Qdrant (`/qdrant/storage`),
* dati CrewAI (`/data`),
* cache modelli condivisa tra più container.

---

## **4. Differenza tra `-v` e `--mount`**

| Caratteristica  | `-v`                   | `--mount`                               |
| --------------- | ---------------------- | --------------------------------------- |
| Sintassi        | più corta              | più leggibile e sicura                  |
| Tipo            | supporta bind e volume | supporta bind, volume e tmpfs           |
| Specificità     | stringa unica          | parametri chiari (type, source, target) |
| Consigliato per | uso rapido             | ambienti complessi o Compose file       |

 In produzione e in file `docker-compose.yml`, **usa sempre `--mount`** o la forma YAML di `volumes:`.

---

## **5. Persistenza di notebook e dati**

Se vuoi eseguire **Jupyter Notebook in container**, devi montare la directory del progetto:

```bash
docker run -it -p 8888:8888 -v %cd%:/notebooks jupyter/base-notebook
```

Apri il browser su `localhost:8888` e troverai tutti i tuoi file locali visibili in `/notebooks`.

Ogni notebook salvato lì resta anche dopo il riavvio del container.

---

## **6. Persistenza dei modelli AI**

Le librerie Hugging Face salvano modelli scaricati in:

```
~/.cache/huggingface
```

Se vuoi evitare di riscaricarli ogni volta, monta la cache come volume persistente:

```bash
docker run -it \
  -v %USERPROFILE%\.cache\huggingface:/root/.cache/huggingface \
  pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime bash
```

Ora ogni container condividerà la stessa cache dei modelli.

---

## **7. Persistenza di database e vector store**

### Qdrant:

```bash
docker run -d \
  -p 6333:6333 \
  -v qdrant_data:/qdrant/storage \
  qdrant/qdrant:v1.10.0
```

Il volume `qdrant_data` mantiene l’indice vettoriale anche dopo il riavvio.

### PostgreSQL (es. per CrewAI logs):

```bash
docker run -d \
  -p 5432:5432 \
  -v pg_data:/var/lib/postgresql/data \
  -e POSTGRES_PASSWORD=crewpass \
  postgres:15
```

---

## **8. Reti personalizzate (`--network`)**

In un progetto AI multi-container, serve spesso far comunicare i servizi tra loro (es. CrewAI → Qdrant → Streamlit).
Per farlo Docker offre le **reti bridge personalizzate**.

### Creare una rete:

```bash
docker network create ai_net
```

### Eseguire container collegati alla stessa rete:

```bash
docker run -d --name qdrant --network ai_net qdrant/qdrant:v1.10.0
docker run -it --name crew_backend --network ai_net python:3.11 bash
```

Ora, da dentro `crew_backend`, puoi connetterti a `qdrant` usando:

```python
from qdrant_client import QdrantClient
client = QdrantClient(host="qdrant", port=6333)
```

 *Non serve conoscere l’IP*, Docker risolve automaticamente i nomi dei container come host DNS.

---

## **9. Esercizi pratici**

### 🔹 **Esercizio 1 – Volume locale**

1. Crea una cartella `C:\docker_test`.
2. Avvia un container con:

   ```bash
   docker run -it -v C:\docker_test:/app python:3.11 bash
   ```
3. Dentro `/app`, crea `test.py` e scrivi qualcosa.
4. Chiudi, verifica che il file sia rimasto nella cartella locale.

---

### 🔹 **Esercizio 2 – Volume Docker**

1. Crea volume:

   ```bash
   docker volume create crew_data
   ```
2. Avvia container:

   ```bash
   docker run -it --mount source=crew_data,target=/data python:3.11 bash
   ```
3. Crea file `/data/persist.txt`.
4. Esci e rimuovi il container.
5. Avvia un nuovo container con lo stesso volume e verifica che il file esista ancora.

---

### 🔹 **Esercizio 3 – Rete condivisa**

1. Crea rete:

   ```bash
   docker network create ai_net
   ```
2. Avvia Qdrant:

   ```bash
   docker run -d --name qdrant --network ai_net qdrant/qdrant:v1.10.0
   ```
3. Avvia Python backend:

   ```bash
   docker run -it --network ai_net python:3.11 bash
   ```
4. Dentro Python:

   ```python
   from qdrant_client import QdrantClient
   client = QdrantClient(host="qdrant", port=6333)
   print(client.get_collections())
   ```

---

## **10. Riepilogo**

| Comando / Opzione               | Funzione                             | Esempio tipico AI                      |
| ------------------------------- | ------------------------------------ | -------------------------------------- |
| `-v host:container`             | Collega cartella locale (bind mount) | Notebook, codice sorgente              |
| `--mount source=...,target=...` | Volume gestito da Docker             | Cache modelli, database, indici Qdrant |
| `docker volume create <name>`   | Crea volume persistente              | `docker volume create crew_data`       |
| `docker network create <name>`  | Crea rete personalizzata             | `docker network create ai_net`         |
| `--network <name>`              | Collega container alla stessa rete   | `--network ai_net`                     |
| `docker volume ls`              | Mostra volumi                        | Verifica storage CrewAI                |
| `docker network ls`             | Mostra reti                          | Debug connessioni tra container        |

---

Docker non è solo “un contenitore”: è un **ecosistema di filesystem, reti e processi**.
Per un AI Engineer, capire come usare `-v`, `--mount` e `--network` significa saper costruire **pipeline CrewAI e RAG persistenti**, scalabili e interconnesse in modo professionale.

---




# **2.1 – Cos’è un Dockerfile**

---

## **1. Definizione**

Un **Dockerfile** è un semplice **file di testo** che contiene **le istruzioni** per costruire un’immagine Docker.

In pratica:

* descrive **passo dopo passo** come creare un ambiente completo (sistema operativo, librerie, codice, comandi d’avvio);
* Docker lo “legge” e, seguendo quelle istruzioni, costruisce un’immagine pronta all’uso;
* quell’immagine può poi essere eseguita su qualsiasi computer o server con Docker installato, **sempre nello stesso modo**.

È, a tutti gli effetti, **la ricetta dell’ambiente di esecuzione**.

---

## **2. Analogia**

Pensa al Dockerfile come a una **ricetta di cucina**:

* Ogni riga è un ingrediente o un’azione (es. “installa Python”, “copia il codice”).
* Docker è il cuoco che la legge e prepara il piatto (l’immagine).
* L’immagine finale è il **piatto pronto** (un ambiente completo e isolato).
* Quando “servi il piatto” (cioè lanci un container), Docker prende quell’immagine e la esegue come un piccolo sistema operativo in miniatura.

---

## **3. Perché serve**

Nel mondo AI o software moderno, lavorare su progetti complessi come:

* **CrewAI** (più agenti, librerie AI, dipendenze pesanti),
* **LangChain** (diverse versioni di pacchetti),
* **Qdrant** o **PostgreSQL** (database separati),
* **Streamlit** o **FastAPI** (frontend e API),

significa dover gestire decine di librerie e ambienti.
Un Dockerfile risolve questo problema: crea un ambiente **standardizzato e replicabile**, che chiunque può ricostruire con un solo comando.

---

## **4. Come funziona**

Docker legge il Dockerfile **dall’alto verso il basso**.
Ogni istruzione crea un “**layer**” dell’immagine: un piccolo pezzo di filesystem.
L’insieme di questi layer forma l’immagine finale.

Esempio semplificato:

```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY . .
RUN pip install -r requirements.txt
CMD ["python", "main.py"]
```

Vediamolo in parole:

1. **FROM python:3.11-slim** → usa come base un sistema Linux con Python 3.11 già installato.
2. **WORKDIR /app** → imposta la directory di lavoro.
3. **COPY . .** → copia i file del progetto nel container.
4. **RUN pip install -r requirements.txt** → installa le librerie.
5. **CMD ["python", "main.py"]** → definisce il comando da eseguire quando il container parte.

Il risultato sarà un’immagine Docker con:

* Python 3.11
* tutte le librerie del progetto
* il codice copiato
* pronta per avviarsi con `python main.py`.

---

## **5. Creare e usare un Dockerfile**

1. Crea un file chiamato `Dockerfile` (senza estensione).
2. Scrivi le istruzioni dentro.
3. Costruisci l’immagine con:

   ```bash
   docker build -t nome-immagine .
   ```

   (`-t` = tag, `.` = directory corrente dove si trova il Dockerfile)
4. Avvia un container da quell’immagine:

   ```bash
   docker run nome-immagine
   ```

Esempio completo:

```bash
docker build -t crewai-backend .
docker run -it crewai-backend
```

---

## **6. Struttura tipica**

Ogni Dockerfile segue un ordine logico:

| Sezione        | Istruzione                 | Funzione                                                  |
| -------------- | -------------------------- | --------------------------------------------------------- |
| Base           | `FROM`                     | Sceglie l’immagine di partenza (es. Python, Node, Ubuntu) |
| Setup          | `RUN`                      | Esegue comandi (es. installazioni, aggiornamenti)         |
| Copia codice   | `COPY` / `ADD`             | Copia file dal computer nel container                     |
| Configurazione | `ENV`, `WORKDIR`, `EXPOSE` | Imposta variabili e directory                             |
| Avvio          | `CMD` / `ENTRYPOINT`       | Definisce il comando di esecuzione                        |

---

## **7. Differenza tra immagine e container**

* Il **Dockerfile** costruisce l’**immagine** (il modello).
* Da quell’immagine si possono creare più **container** (le istanze in esecuzione).

Esempio:

```bash
docker build -t crewai:latest .
docker run -d --name crew1 crewai:latest
docker run -d --name crew2 crewai:latest
```

→ stesso ambiente, due container indipendenti.

---

## **8. Esercizio pratico**

1. Crea una nuova cartella:

   ```
   docker_test/
   ├── Dockerfile
   └── main.py
   ```
2. In `main.py` scrivi:

   ```python
   print("Hello from Docker!")
   ```
3. In `Dockerfile`:

   ```dockerfile
   FROM python:3.11-slim
   WORKDIR /app
   COPY . .
   CMD ["python", "main.py"]
   ```
4. Costruisci e avvia:

   ```bash
   docker build -t hello-docker .
   docker run hello-docker
   ```

**Output:**

```
Hello from Docker!
```

Ora hai creato la tua prima immagine personalizzata.

---

## **9. In sintesi**

| Concetto             | Descrizione                                                   |
| -------------------- | ------------------------------------------------------------- |
| Dockerfile           | File di testo con istruzioni per costruire un ambiente Docker |
| Immagine             | L’ambiente creato a partire dal Dockerfile                    |
| Container            | L’istanza in esecuzione dell’immagine                         |
| `docker build`       | Crea l’immagine                                               |
| `docker run`         | Esegue il container                                           |
| `FROM`               | Base del sistema                                              |
| `RUN`, `COPY`, `CMD` | Istruzioni principali                                         |

---

Un Dockerfile è quindi **il cuore di ogni progetto containerizzato**.
Nei prossimi punti impareremo:

* come ottimizzarlo per AI (ridurre peso e tempi di build),
* come gestire dipendenze, cache e volumi,
* e come scrivere Dockerfile modulari per pipeline CrewAI reali.

---


# **2.2 – Istruzioni fondamentali del Dockerfile**

---

## **1. `FROM` – l’immagine di base**

È **sempre la prima riga** di un Dockerfile.
Serve per specificare **da quale sistema operativo o ambiente di partenza** costruire la tua immagine.

Esempi:

```dockerfile
FROM python:3.11-slim
```

→ parte da Debian “slim” con Python 3.11 già installato.

```dockerfile
FROM pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime
```

→ parte da un ambiente già pronto per PyTorch con CUDA e cuDNN.

### Regole:

* puoi usare **immagini ufficiali** (es. python, node, ubuntu) o personalizzate;
* è possibile **cambiare base** in base al progetto (CPU-only o GPU);
* ogni immagine base contiene già un piccolo sistema Linux.

 *Best practice per AI:*

* per ambienti CPU: `python:3.11-slim`;
* per GPU NVIDIA: `pytorch/pytorch:<ver>-cuda<xx>-runtime`;
* per GPU AMD: `rocm/pytorch:latest`.

---

## **2. `WORKDIR` – directory di lavoro**

Definisce **la cartella in cui verranno eseguiti tutti i comandi successivi** (come `RUN`, `COPY`, `CMD`, ecc.).

Esempio:

```dockerfile
WORKDIR /app
```

→ Tutti i comandi successivi verranno eseguiti come se fossi dentro `/app`.

Puoi definirne più d’uno:

```dockerfile
WORKDIR /usr/src
WORKDIR /app
```

→ il secondo sovrascrive il primo (come un `cd`).

 *Best practice:*
Usa sempre `WORKDIR` e **non** `/` o directory di sistema per scrivere i tuoi file.

---

## **3. `COPY` – copia file dall’host al container**

Serve per trasferire file e cartelle dal tuo computer (host) all’interno dell’immagine.

Esempio:

```dockerfile
COPY . .
```

→ copia tutti i file della directory corrente nel container (nella `WORKDIR`).

Puoi anche essere più specifico:

```dockerfile
COPY requirements.txt .
COPY src/ ./src
```

 *Suggerimenti:*

* escludi file inutili con `.dockerignore` (es. `.git`, `__pycache__`, `venv`, `data/`);
* mantieni ordine: prima copia i file di dipendenze (`requirements.txt`), poi il codice.

---

## **4. `RUN` – esegue comandi durante la build**

`RUN` viene eseguito **mentre si costruisce l’immagine**, non quando il container parte.
Serve per installare librerie, creare directory, aggiornare pacchetti, ecc.

Esempi:

```dockerfile
RUN apt-get update && apt-get install -y git
RUN pip install --no-cache-dir -r requirements.txt
```

Ogni `RUN` crea un **nuovo layer** dell’immagine.

 *Ottimizzazione:*

* combina più comandi con `&&` per ridurre i layer;
* pulisci la cache di apt per alleggerire l’immagine:

  ```dockerfile
  RUN apt-get update && apt-get install -y git && rm -rf /var/lib/apt/lists/*
  ```


---

## **5. `ENV` – definisce variabili d’ambiente**

Le variabili definite con `ENV` sono visibili **dentro il container** a runtime.
Molto utile per chiavi API, configurazioni o impostazioni di Python.

Esempi:

```dockerfile
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV OPENAI_API_KEY="sk-xxxxx"
```

Puoi anche dichiararle tutte insieme:

```dockerfile
ENV PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    PIP_DISABLE_PIP_VERSION_CHECK=1
```

 *Best practice per AI:*

* imposta sempre `PYTHONUNBUFFERED=1` → output immediato nei log;
* evita di scrivere chiavi sensibili nel Dockerfile → passa le variabili con `-e` nel `docker run`.

Esempio runtime:

```bash
docker run -e OPENAI_API_KEY=$OPENAI_API_KEY crewai-backend
```

---

## **6. `EXPOSE` – documenta le porte usate**

Serve per **indicare quale porta interna del container** viene usata dal servizio.
Non apre realmente la porta, ma aiuta Docker a sapere come mappare correttamente.

Esempio:

```dockerfile
EXPOSE 8080
```

→ Il container comunica sulla porta 8080.
Poi, quando lo lanci:

```bash
docker run -p 8080:8080 crewai-backend
```

la porta 8080 del container è raggiungibile come `localhost:8080`.

 *Best practice:*

* API backend: 8000 o 8080
* Qdrant: 6333
* Streamlit: 8501

---

## **7. `CMD` – comando di avvio del container**

È il comando che viene eseguito **quando il container parte**.
Ogni immagine può avere **un solo CMD**.

Esempio:

```dockerfile
CMD ["python", "main.py"]
```

Alternative:

```dockerfile
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8080"]
CMD ["streamlit", "run", "ui/Home.py", "--server.port", "8501"]
```

 *Regole:*

* se nel `docker run` specifichi un comando manuale, questo **sovrascrive** il CMD;

  ```bash
  docker run myimage python other_script.py
  ```
* il CMD deve sempre essere l’ultimo comando del Dockerfile;
* usa la sintassi **JSON array** (non shell) per evitare errori di parsing.

---

## **8. `ENTRYPOINT` – comando fisso di avvio**

Simile a `CMD`, ma **non può essere sovrascritto** facilmente.
Serve per rendere “eseguibile” il container.

Esempio:

```dockerfile
ENTRYPOINT ["python"]
CMD ["main.py"]
```

→ di default esegue `python main.py`, ma puoi fare:

```bash
docker run myimage other.py
```

→ eseguirà `python other.py`.

 *Regola generale:*

* usa `ENTRYPOINT` se vuoi rendere il container eseguibile (CLI tool, script);
* usa `CMD` per applicazioni (server, API).

---

## **9. Altri comandi utili (cenni rapidi)**

| Comando       | Funzione                                                            |
| ------------- | ------------------------------------------------------------------- |
| `ADD`         | Come `COPY`, ma può scaricare URL o scompattare archivi             |
| `LABEL`       | Aggiunge metadati all’immagine (autore, versione)                   |
| `USER`        | Imposta l’utente che esegue i comandi (non root per sicurezza)      |
| `ARG`         | Variabili disponibili solo durante la build (es. `ARG PY_VER=3.11`) |
| `ONBUILD`     | Comandi che si attivano solo in immagini derivate                   |
| `HEALTHCHECK` | Controlla lo stato del container periodicamente                     |

---

## **10. Esercizio pratico: costruire un Dockerfile completo**

1. Crea una cartella `myapp/`

   ```
   myapp/
   ├── Dockerfile
   ├── requirements.txt
   └── app.py
   ```

2. In `requirements.txt`:

   ```
   fastapi==0.114.0
   uvicorn==0.30.6
   ```

3. In `app.py`:

   ```python
   from fastapi import FastAPI
   app = FastAPI()

   @app.get("/")
   def hello():
       return {"msg": "Hello from Docker!"}
   ```

4. In `Dockerfile`:

   ```dockerfile
   FROM python:3.11-slim
   WORKDIR /app
   COPY requirements.txt .
   RUN pip install --no-cache-dir -r requirements.txt
   COPY . .
   EXPOSE 8080
   CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8080"]
   ```

5. Costruisci e lancia:

   ```bash
   docker build -t fastapi-demo .
   docker run -p 8080:8080 fastapi-demo
   ```

6. Apri [http://localhost:8080](http://localhost:8080)
   → dovresti vedere:
   `{"msg": "Hello from Docker!"}`

---

## **11. Riepilogo**

| Istruzione   | Scopo                      | Esempio                               |
| ------------ | -------------------------- | ------------------------------------- |
| `FROM`       | Base dell’immagine         | `FROM python:3.11-slim`               |
| `WORKDIR`    | Directory di lavoro        | `WORKDIR /app`                        |
| `COPY`       | Copia file nel container   | `COPY . .`                            |
| `RUN`        | Esegue comandi di build    | `RUN pip install -r requirements.txt` |
| `ENV`        | Imposta variabili ambiente | `ENV PYTHONUNBUFFERED=1`              |
| `EXPOSE`     | Documenta porte            | `EXPOSE 8080`                         |
| `CMD`        | Comando di avvio           | `CMD ["python", "main.py"]`           |
| `ENTRYPOINT` | Comando fisso di avvio     | `ENTRYPOINT ["python"]`               |

---




# **2.3 – Multi-Stage Build: spiegazione semplice e chiara**

---

## **1. Il problema da cui nasce**

Quando crei un’immagine Docker, spesso hai **due momenti distinti**:

1. **La costruzione (build)**
   dove installi compilatori, scarichi repository da Git, compili estensioni, ecc.
   → serve molta roba (pacchetti di sviluppo, tool, file temporanei).

2. **L’esecuzione (runtime)**
   dove ti serve solo il risultato finale: il tuo programma e le librerie già pronte.
   → tutto il resto (tool, cache, file temporanei) è inutile e appesantisce l’immagine.

---

###  Senza multi-stage

Se fai tutto nello stesso Dockerfile, ti ritrovi un’immagine **enorme** e piena di file inutili.

Esempio:

```dockerfile
FROM python:3.11-slim
RUN apt-get update && apt-get install -y build-essential git
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
CMD ["python", "main.py"]
```

Risultato:

* immagine da **2-3 GB**,
* dentro ci sono compilatori, header file, cache pip…
* e tutto ciò non serve più a runtime.

---

## **2. L’idea del multi-stage build**

Docker ti permette di dividere la build in **più “fasi” (stages)**.
Ogni stage ha la sua immagine base e il suo ambiente.
Alla fine puoi **prendere solo quello che ti serve** e buttar via tutto il resto.

In pratica:

* uno **stage builder** fa tutto il lavoro pesante (installa, compila, crea file).
* uno **stage finale (runtime)** parte pulito e riceve **solo i file utili** dal builder.

---

###  Esempio semplice

```dockerfile
# STAGE 1 – Builder
FROM python:3.11-slim AS builder
WORKDIR /build
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# STAGE 2 – Runtime
FROM python:3.11-slim AS runtime
WORKDIR /app
COPY --from=builder /build /app
COPY . .
CMD ["python", "main.py"]
```

Spiegazione:

* La prima parte (`AS builder`) crea un’immagine temporanea chiamata *builder*.
* Installa lì tutte le dipendenze.
* La seconda parte (`FROM ... AS runtime`) crea l’immagine finale.
* `COPY --from=builder` prende solo ciò che serve dal primo stage.
* Il risultato finale è **pulito, piccolo e pronto all’uso**.

---

## **3. Cosa succede dentro Docker**

Quando Docker costruisce questo file:

1. Crea il primo stage (`builder`), installa tutto.
2. Poi **scarta** quell’immagine, ma conserva i file copiati.
3. Costruisce la seconda immagine (`runtime`), che contiene solo quei file.

→ Risultato: la seconda immagine non ha compilatori, cache o file temporanei.

---

## **4. Vantaggi pratici**

| Vantaggio                | Descrizione                                       |
| ------------------------ | ------------------------------------------------- |
| **Immagini più leggere** | Eviti di includere tool di build e cache pip      |
| **Build più pulite**     | Separi logica di build da logica di esecuzione    |
| **Più sicurezza**        | Meno tool e pacchetti = meno vulnerabilità        |
| **Facile da mantenere**  | Ogni fase ha uno scopo chiaro                     |
| **Ottimo per CI/CD**     | Puoi riusare gli stessi stage in pipeline diverse |

---

## **5. Esempio reale – Python con librerie AI**

Supponiamo di voler costruire un’immagine per un progetto CrewAI o LangChain.
Serve scaricare molte dipendenze Python, alcune anche da Git.

Ecco come useresti un multi-stage build semplice:

```dockerfile
# FASE 1 — Builder
FROM python:3.11-slim AS builder
WORKDIR /build

# Installa git e altri strumenti solo qui
RUN apt-get update && apt-get install -y --no-install-recommends git build-essential

# Copia le dipendenze e installale in una directory temporanea
COPY requirements.txt .
RUN pip install --target=/build/deps -r requirements.txt

# FASE 2 — Runtime
FROM python:3.11-slim AS runtime
WORKDIR /app

# Copia solo i file delle librerie già installate
COPY --from=builder /build/deps /usr/local/lib/python3.11/site-packages

# Copia il codice dell’app
COPY . .

CMD ["python", "main.py"]
```

 Risultato:

* L’immagine finale contiene solo Python + le librerie installate + il tuo codice.
* Tutti i tool di build e la cache pip **restano nel builder e vengono scartati**.

---

## **6. Multi-stage = più di due fasi**

Puoi avere anche 3 o più fasi se vuoi separare meglio i compiti.
Esempio tipico per un’app AI con API:

1. **builder** → prepara le dipendenze (pip o poetry).
2. **api-builder** → costruisce file statici o script ottimizzati.
3. **runtime** → esegue l’app (FastAPI o Streamlit).

Docker ti permette di copiare file da qualunque fase:

```dockerfile
COPY --from=api-builder /dist /app
```

---

## **7. Come dare un nome alle fasi**

Ogni fase ha un nome definito da `AS`.
Esempio:

```dockerfile
FROM python:3.11-slim AS builder
...
FROM python:3.11-slim AS runtime
COPY --from=builder /build /app
```

Il nome è utile per dire a Docker **da quale stage copiare i file**.

---

## **8. Esercizio pratico – Creiamo due immagini a confronto**

### A) Dockerfile semplice (senza multi-stage)

```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN apt-get update && apt-get install -y git && pip install -r requirements.txt
COPY . .
CMD ["python", "main.py"]
```

### B) Dockerfile multi-stage

```dockerfile
FROM python:3.11-slim AS builder
WORKDIR /build
RUN apt-get update && apt-get install -y git
COPY requirements.txt .
RUN pip install --target=/build/deps -r requirements.txt

FROM python:3.11-slim
WORKDIR /app
COPY --from=builder /build/deps /usr/local/lib/python3.11/site-packages
COPY . .
CMD ["python", "main.py"]
```

### Confronto

| Aspetto              | A) Singolo stage | B) Multi-stage         |
| -------------------- | ---------------- | ---------------------- |
| Dimensione immagine  | 2-3 GB           | ~800 MB                |
| Contiene build-tools | ✅ sì             | ❌ no                   |
| Sicurezza            | minore           | maggiore               |
| Portabilità          | limitata         | alta                   |
| Tempo rebuild        | più lento        | più rapido (cache pip) |

---

## **9. Come si costruisce**

Comando identico:

```bash
docker build -t myapp:latest .
```

Docker capisce automaticamente che ci sono più fasi e costruisce solo l’ultima (salvando cache intermedia).

---

## **10. Cosa ricordare**

| Concetto                                | Spiegazione                                   |
| --------------------------------------- | --------------------------------------------- |
| Ogni `FROM` crea una nuova fase         | Le fasi possono avere nomi (`AS builder`)     |
| `COPY --from=`                          | Copia file da una fase all’altra              |
| Lo stage finale è l’immagine che rimane | Tutto il resto viene eliminato                |
| Obiettivo                               | separare build “pesante” da runtime “leggero” |
| Beneficio principale                    | immagini più piccole, sicure e veloci         |

---

 **In sintesi**

* Il multi-stage non cambia cosa fa Docker, ma **come lo organizza**.
* È il modo corretto di costruire immagini **professionali**, soprattutto nel mondo AI.
* Ti permette di preparare ambienti complessi (Torch, Transformers, CrewAI, Qdrant) e distribuire solo ciò che serve per l’esecuzione.

---





# **2.4 – ENTRYPOINT per comandi complessi**

---

## **1. A cosa serve `ENTRYPOINT`**

Quando costruisci un’immagine, devi dire a Docker **che cosa fare quando il container parte**.
Puoi farlo in due modi:

* con `CMD`
* con `ENTRYPOINT`

Entrambi indicano **il comando di avvio**, ma con una differenza importante:

| Comando      | Può essere sovrascritto da `docker run` | Tipico uso                                     |
| ------------ | --------------------------------------- | ---------------------------------------------- |
| `CMD`        | ✅ sì                                    | eseguire un’app o uno script semplice          |
| `ENTRYPOINT` | ❌ no (di default)                       | rendere il container un *programma* eseguibile |

 `ENTRYPOINT` rende l’immagine “comportarsi” come un comando.
Ad esempio, se hai un tool AI che esegue analisi o scraping, vuoi lanciare:

```bash
docker run my-ai-analyzer input.txt
```

e far sì che il container esegua qualcosa tipo:

```bash
python analyze.py input.txt
```

---

## **2. Due sintassi possibili**

### a) **Exec form (raccomandata)**

```dockerfile
ENTRYPOINT ["python", "main.py"]
```

→ Docker esegue direttamente il processo come se fosse nativo (senza shell).

### b) **Shell form**

```dockerfile
ENTRYPOINT python main.py
```

→ viene eseguito dentro `/bin/sh -c`, utile se ti servono operatori shell (`&&`, `|`, `;`), ma meno sicuro e meno efficiente.

 *Best practice*: usa **exec form** (lista JSON) per applicazioni reali, shell form solo per script d’avvio complessi.

---

## **3. ENTRYPOINT + CMD = combinazione potente**

Puoi usare **entrambi**.
Docker combina `ENTRYPOINT` e `CMD`:

* `ENTRYPOINT` definisce **il programma principale**,
* `CMD` definisce **gli argomenti di default**.

Esempio:

```dockerfile
ENTRYPOINT ["python", "main.py"]
CMD ["--help"]
```

→ se lanci:

```bash
docker run myapp
```

esegue:
`python main.py --help`

Ma se lanci:

```bash
docker run myapp --version
```

esegue:
`python main.py --version`

> Docker sostituisce **solo gli argomenti del CMD**, non l’ENTRYPOINT.

---


## **4. ENTRYPOINT + script Bash per comandi complessi**

Spesso ti serve eseguire più cose all’avvio (es. inizializzare modelli, verificare database, poi avviare CrewAI).
In questo caso puoi usare uno **script shell** come entrypoint.

### a) Dockerfile

```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY . .
RUN pip install crewai qdrant-client
COPY entrypoint.sh /entrypoint.sh
RUN chmod +x /entrypoint.sh
ENTRYPOINT ["/entrypoint.sh"]
```

### b) `entrypoint.sh`

```bash
#!/bin/bash
set -e  # interrompe se c’è un errore

echo "🧠 Inizializzazione CrewAI..."
python setup_models.py

echo "🗄️  Controllo connessione Qdrant..."
python check_qdrant.py

echo "🚀 Avvio server principale..."
exec python main.py "$@"
```

 `exec` è importante: sostituisce il processo Bash con Python, evitando che Docker perda i log o il PID del processo principale.

---

## **5. ENTRYPOINT con comandi concatenati**

Puoi anche combinare comandi multipli direttamente nel Dockerfile, ma solo se necessario:

```dockerfile
ENTRYPOINT ["/bin/bash", "-c", "python prepare.py && python main.py"]
```

Tuttavia, **non è consigliato** per progetti complessi — molto meglio usare uno script dedicato come visto sopra.

---

## **6. Personalizzare i parametri in `docker run`**

Con `ENTRYPOINT`, puoi passare argomenti extra da linea di comando.

Esempio:

```dockerfile
ENTRYPOINT ["python", "main.py"]
CMD ["--help"]
```

Puoi cambiare i parametri:

```bash
docker run myapp --input=data.txt --verbose
```

→ Docker esegue `python main.py --input=data.txt --verbose`

---

## **7. Esempio complesso: FastAPI + CrewAI orchestrato**

Ecco un esempio realistico per un progetto AI con più servizi:

```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
COPY entrypoint.sh /entrypoint.sh
RUN chmod +x /entrypoint.sh  #change mode to x, executable
ENTRYPOINT ["/entrypoint.sh"]
```

`entrypoint.sh`:

```bash
#!/bin/bash
set -e

echo " Avvio Qdrant (in background)..."
docker-entrypoint.sh qdrant &

echo " Avvio CrewAI Server..."
exec uvicorn app.main:app --host 0.0.0.0 --port 8080
```

In Compose:

```yaml
services:
  crewai:
    build: .
    ports: ["8080:8080"]
    depends_on: [qdrant]
```

> Lo script fa partire prima Qdrant, poi CrewAI, in modo ordinato e monitorabile.

---

## **8. Differenze tra ENTRYPOINT e CMD (recap)**

| Aspetto                         | `ENTRYPOINT`                           | `CMD`                                    |
| ------------------------------- | -------------------------------------- | ---------------------------------------- |
| Scopo                           | definisce **il programma principale**  | definisce **gli argomenti di default**   |
| Sovrascrizione con `docker run` | ❌ no (a meno di usare `--entrypoint`)  | ✅ sì                                     |
| Sintassi preferita              | JSON array                             | JSON array                               |
| Tipico uso                      | CLI, script di startup, orchestrazione | impostare default o parametri            |
| Posizione                       | 1 sola per Dockerfile                  | 1 sola (spesso combinata con ENTRYPOINT) |

---




# **2.5 – Caching layer e `.dockerignore` nei progetti CrewAI**

---

## **1. Il problema reale**

I progetti AI (e in particolare CrewAI) hanno due caratteristiche:

1. **Dipendenze pesanti**
   Torch, Transformers, LangChain, Qdrant-client, OpenAI SDK, ecc.
   Ogni `pip install` può impiegare minuti e scaricare centinaia di MB.

2. **File locali numerosi e inutili per la build**
   cartelle `.git`, `.venv`, `models/`, `__pycache__/`, `data/`, `.env` → inutili nell’immagine.

Senza caching e `.dockerignore`, ogni modifica nel codice fa **ripartire tutto da zero**,
ricostruendo anche le librerie pesanti e copiando file superflui.
Il risultato?

* tempi di build lunghi (anche >10 minuti),
* immagini da diversi GB,
* scarsa portabilità.

---

## **2. Cos’è la cache di Docker**

Docker costruisce un’immagine **a layer**: ogni istruzione (`FROM`, `RUN`, `COPY`, ecc.) crea un “pezzo” del filesystem.

Esempio:

```dockerfile
FROM python:3.11-slim
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
```

* Se **requirements.txt non cambia**, Docker può riutilizzare la cache del layer `RUN pip install`.
* Ma se lo modifichi o se `COPY . .` viene prima, **Docker rigenera tutto da zero**.

 Quindi **l’ordine delle istruzioni conta moltissimo** per la cache.

---

## **3. Best practice per la cache (ordine corretto)**

Ordina sempre così:

```dockerfile
FROM python:3.11-slim
WORKDIR /app

# 1️⃣ Copia SOLO requirements (cambia raramente)
COPY requirements.txt .

# 2️⃣ Installa dipendenze (caching pip)
RUN --mount=type=cache,target=/root/.cache/pip \
    pip install --upgrade pip \
 && pip install --no-cache-dir -r requirements.txt

# 3️⃣ Copia il codice del progetto (cambia spesso)
COPY . .
```

* **`COPY requirements.txt .`**: viene eseguito solo se requirements cambia → cache efficace.
* **`COPY . .`**: viene eseguito dopo → evita di invalidare l’installazione pip per ogni piccolo update del codice.

---


## **4. Cos’è `.dockerignore` e perché è fondamentale**

`.dockerignore` funziona come `.gitignore`:
indica a Docker **quali file NON copiare nel contesto di build** (cioè la directory che Docker invia al demone).

Se Docker deve analizzare 5 GB di dati in `data/` o `models/`,
li invierà tutti ogni volta, rallentando la build anche se non servono.

### Esempio tipico

```
.git
__pycache__/
*.pyc
*.pyo
*.pyd
venv/
.env
.cache/
dist/
build/
models/
data/
node_modules/
outputs/
logs/
```

 *Note per CrewAI*:

* **`models/`**: non includere mai modelli HF o checkpoint pesanti → montali come volume.
* **`data/`**: includila solo se serve nel runtime (dataset statici o demo).
* **`.env`**: contiene chiavi API → escludilo sempre per sicurezza.
* **`__pycache__/`** e `.pyc` → inutili.
* **`node_modules/`** → solo se hai interfaccia React/Streamlit.

---

## **5. Esempio reale di `.dockerignore` per progetto CrewAI completo**

```
# File temporanei e di sistema
__pycache__/
*.pyc
*.pyo
*.pyd
*.log
.DS_Store

# Ambienti locali
venv/
.env
.venv/
.cache/
__pypackages__/

# Repositori e build
.git
.gitignore
build/
dist/
*.egg-info/

# Dati pesanti
models/
data/
outputs/
logs/
*.csv
*.zip
*.tar.gz

# Frontend
node_modules/
npm-debug.log
yarn-error.log

# IDE
.vscode/
.idea/
```

 In questo modo Docker copia **solo il codice e i file essenziali**,
riducendo drasticamente la dimensione del contesto di build.

---

## **6. Come testare l’effetto del `.dockerignore`**

Puoi vedere quali file vengono effettivamente inclusi nel contesto:

```bash
docker build -t crewai-backend .
```

Durante la build, Docker stampa:

```
Sending build context to Docker daemon  12.5MB
```

→ se prima erano 2GB e ora 12MB, `.dockerignore` sta funzionando.

---

## **7. Cache + .dockerignore = performance reale**

| Azione                             | Senza ottimizzazione | Con cache + .dockerignore |
| ---------------------------------- | -------------------- | ------------------------- |
| Prima build                        | 6–8 minuti           | 6–8 minuti                |
| Rebuild dopo piccolo cambio codice | 6–8 minuti           | 15–30 secondi             |
| Dimensione immagine                | ~2–3 GB              | ~800 MB                   |
| Contesto inviato a Docker          | >1 GB                | ~10–20 MB                 |
| Rischio di leak `.env`             | alto                 | nullo                     |

---

## **8. Montare cache persistenti per modelli**

Per modelli AI pesanti (es. Transformers, Sentence-Embeddings), **non scaricarli ogni volta**.
Usa un volume per la cache HuggingFace:

```bash
docker run -p 8080:8080 \
  -v $PWD/models:/root/.cache/huggingface \
  crewai-backend
```

Così:

* i modelli vengono scaricati una sola volta;
* i container successivi li riutilizzano;
* l’immagine rimane leggera e veloce da distribuire.

---

## **9. Errori comuni (e come evitarli)**

| Errore                      | Causa                                   | Soluzione                     |
| --------------------------- | --------------------------------------- | ----------------------------- |
| Ogni build reinstalla tutto | `COPY . .` prima del `pip install`      | inverti l’ordine              |
| Build lentissima            | contesto enorme (manca `.dockerignore`) | crea `.dockerignore` completo |
| Modelli dentro l’immagine   | copia di `models/` o `data/`            | usa volume montato            |
| `.env` incluso              | non ignorato                            | aggiungilo a `.dockerignore`  |
| Cache pip non usata         | mancanza `--mount=type=cache`           | abilita BuildKit              |

---

## **10. In sintesi**

| Obiettivo                           | Soluzione                                        |
| ----------------------------------- | ------------------------------------------------ |
| Evitare reinstallazioni inutili     | Copia requirements prima del codice              |
| Riutilizzare librerie già scaricate | Usa `--mount=type=cache,target=/root/.cache/pip` |
| Evitare file inutili nel contesto   | Usa `.dockerignore` completo                     |
| Evitare modelli nell’immagine       | Monta `~/.cache/huggingface` come volume         |
| Build più veloci e pulite           | Struttura Dockerfile con caching consapevole     |

---

## **11. Prova pratica (5 minuti)**

1. Crea un file `Dockerfile` con l’ordine corretto e cache pip.
2. Crea `.dockerignore` con le regole sopra.
3. Fai due build:

   ```bash
   docker build -t test-cache .
   docker build -t test-cache .
   ```
4. Osserva i tempi: la seconda build deve essere **istantanea** se non hai modificato `requirements.txt`.

---



# 2.6 – Reti per microservizi AI

## (FastAPI backend ↔ Qdrant ↔ Streamlit UI)

## 1) Concetto chiave: rete bridge + DNS interno

* I container **sulla stessa rete Docker** si vedono per **nome di servizio** (DNS interno).
* Non serve conoscere l’IP: dal backend puoi chiamare `http://qdrant:6333` invece di `http://172.18.0.3:6333`.
* Esporre porte sull’host (`-p`) serve solo se **vuoi accedere da fuori** (es. browser).

---

## 2) Setup “a mano” (senza Compose)

### Crea una rete dedicata

```bash
docker network create ai_net
```

### Avvia Qdrant su quella rete

```bash
docker run -d --name qdrant \
  --network ai_net \
  -p 6333:6333 \
  -v qdrant_data:/qdrant/storage \
  qdrant/qdrant:v1.10.0
```

### Avvia il backend FastAPI (CrewAI) sulla stessa rete

```bash
docker run -d --name backend \
  --network ai_net \
  -p 8080:8080 \
  -e QDRANT_HOST=qdrant \
  -e QDRANT_PORT=6333 \
  myorg/crewai-backend:latest
```

Nel codice (Python):

```python
import os
from qdrant_client import QdrantClient

client = QdrantClient(
  host=os.getenv("QDRANT_HOST", "qdrant"),
  port=int(os.getenv("QDRANT_PORT", 6333))
)
```

### Avvia Streamlit UI collegata al backend

```bash
docker run -d --name ui \
  --network ai_net \
  -p 8501:8501 \
  -e API_BASE_URL=http://backend:8080 \
  myorg/streamlit-ui:latest
```

Nella UI:

```python
import os, requests
API = os.getenv("API_BASE_URL", "http://backend:8080")
resp = requests.get(f"{API}/health").json()
```

> Risultato: i tre container **si risolvono per nome** (qdrant, backend, ui) e parlano tra loro senza dover conoscere IP.

---

# **3.1 – Introduzione a Docker Compose per AI stacks**

---

## **1. Perché esiste Docker Compose**

Quando lavori con Docker, ogni container è **isolato e indipendente**.

Esempio:

* Un container ospita il **backend FastAPI** (CrewAI).
* Un altro container ospita **Qdrant**, il database vettoriale.
* Un altro ancora la **UI Streamlit**.

Con `docker run`, dovresti avviarli tutti **a mano**, collegarli a una **rete**, assegnare **porte**, gestire **volumi**, variabili, dipendenze, e ricordarti tutti i parametri ogni volta.

Esempio di quanto diventa scomodo:

```bash
docker network create ai_net

docker run -d --name qdrant \
  --network ai_net \
  -p 6333:6333 \
  -v qdrant_data:/qdrant/storage \
  qdrant/qdrant:v1.10.0

docker run -d --name backend \
  --network ai_net \
  -p 8080:8080 \
  -e QDRANT_HOST=qdrant \
  -e QDRANT_PORT=6333 \
  myorg/crewai-backend:latest

docker run -d --name ui \
  --network ai_net \
  -p 8501:8501 \
  -e API_BASE_URL=http://backend:8080 \
  myorg/streamlit-ui:latest
```

Funziona, ma è **lungo, fragile e ripetitivo**.
Serve un modo per **descrivere tutto questo in un solo file**, leggibile e versionabile.
Quel file è **`docker-compose.yml`**.

---

## **2. Cos’è Docker Compose**

Docker Compose è uno **strumento di orchestrazione locale**.
Serve per **definire e gestire più container come un’unica applicazione**.

In pratica:

* Scrivi **tutta la configurazione** (immagini, porte, variabili, volumi, reti, dipendenze)
* in un file YAML chiamato `docker-compose.yml`
* e poi avvii tutto con **un solo comando**:

```bash
docker compose up
```

Docker Compose:

1. Crea automaticamente la rete interna tra i servizi.
2. Crea i volumi definiti nel file.
3. Costruisce le immagini se hai indicato `build:`.
4. Lancia i container nell’ordine giusto (`depends_on`).
5. Permette di spegnere tutto con un solo `docker compose down`.

---

## **3. Struttura logica di un file `docker-compose.yml`**

Un file Compose segue questa **struttura ad albero**:

```yaml
version: "3.9"       # (opzionale)
services:            # i container dell'applicazione
  <nome_servizio>:
    image: ...       # oppure build: ...
    ports:
    environment:
    volumes:
    depends_on:
volumes:              # volumi persistenti
networks:             # (opzionale) reti personalizzate
```

Ogni **servizio** rappresenta un container.
Ogni container può avere:

* variabili d’ambiente,
* porte esposte,
* volumi da montare,
* dipendenze da altri servizi.

---

## **4. Esempio reale per uno stack AI**

Immagina di avere un progetto con:

* **CrewAI backend** (FastAPI),
* **Qdrant** (vector DB),
* **Streamlit UI**.

Ecco come lo descriveresti:

```yaml
version: "3.9"

services:
  qdrant:
    image: qdrant/qdrant:v1.10.0
    ports:
      - "6333:6333"
    volumes:
      - qdrant_data:/qdrant/storage
    healthcheck:
      test: ["CMD", "wget", "-qO-", "http://localhost:6333/healthz"]
      interval: 10s
      timeout: 3s
      retries: 5

  backend:
    build: ./backend
    ports:
      - "8080:8080"
    environment:
      - QDRANT_HOST=qdrant
      - QDRANT_PORT=6333
    volumes:
      - ./config:/app/config:ro
      - ./models:/root/.cache/huggingface
    depends_on:
      qdrant:
        condition: service_healthy

  ui:
    build: ./ui
    ports:
      - "8501:8501"
    environment:
      - API_BASE_URL=http://backend:8080
    depends_on:
      - backend

volumes:
  qdrant_data:
```

---

## **5. Spiegazione riga per riga**

### 🔹 `version: "3.9"`

Serve per indicare la versione dello schema Compose.
Nelle versioni recenti è **opzionale**: Compose la riconosce automaticamente.

---

### 🔹 `services:`

È la sezione principale.
Ogni chiave al suo interno rappresenta un container (un servizio indipendente).

---

### 🔹 `qdrant:`

Il nome del servizio.
Diventa anche **hostname interno**: il backend potrà connettersi a `http://qdrant:6333`.

---

### 🔹 `image: qdrant/qdrant:v1.10.0`

Specifica quale immagine Docker usare.
Può essere una:

* immagine pubblica (`python:3.11`, `qdrant/qdrant`)
* immagine privata (`myorg/backend:latest`)
* o una da costruire localmente con `build:`.

---

### 🔹 `build: ./backend`

Indica che Compose deve costruire l’immagine a partire dal Dockerfile nella cartella `backend/`.

Se hai già pubblicato la tua immagine (es. su Docker Hub o un registry interno), puoi usare:

```yaml
image: myorg/crewai-backend:1.0
```

---

### 🔹 `ports:`

Serve per **mappare** le porte del container verso l’host.

```yaml
ports:
  - "8080:8080"  # (host:container)
```

Esempio:
`localhost:8080` → porta 8080 dentro il container backend.
Puoi aprire la tua UI su `localhost:8501` o le API su `localhost:8080`.

---

### 🔹 `environment:`

Definisce variabili d’ambiente dentro il container.

Esempio:

```yaml
environment:
  - QDRANT_HOST=qdrant
  - QDRANT_PORT=6333
```

Nel codice Python:

```python
import os
os.getenv("QDRANT_HOST")  # => "qdrant"
```

Puoi anche leggerle da un file `.env` esterno (Compose lo supporta nativamente).

---

### 🔹 `volumes:`

Serve per **montare directory persistenti** o **cartelle locali** nel container.

Tipi di volume:

* **Bind mount**: collega una cartella locale.

  ```yaml
  - ./config:/app/config:ro
  ```

  → leggi i file YAML di CrewAI dal tuo computer (solo lettura).

* **Volume gestito da Docker**:

  ```yaml
  - qdrant_data:/qdrant/storage
  ```

  → storage persistente mantenuto da Docker (non sparisce a ogni `down`).

La sezione finale:

```yaml
volumes:
  qdrant_data:
```

crea il volume se non esiste.

---

### 🔹 `depends_on:`

Definisce le **dipendenze di avvio**.
Docker avvia prima i servizi da cui dipendi.

Esempio:

```yaml
depends_on:
  qdrant:
    condition: service_healthy
```

→ il backend parte **solo quando Qdrant risponde al suo healthcheck**.

---

### 🔹 `healthcheck:`

Serve per dire a Docker come verificare che un servizio sia “vivo”.

Esempio:

```yaml
healthcheck:
  test: ["CMD", "wget", "-qO-", "http://localhost:6333/healthz"]
  interval: 10s
  timeout: 3s
  retries: 5
```

Docker controlla Qdrant ogni 10 secondi e aggiorna il suo stato interno (healthy/unhealthy).

---

### 🔹 `volumes:` (sezione finale)

Qui definisci **tutti i volumi gestiti da Docker**.
Ogni voce è un volume persistente.

```yaml
volumes:
  qdrant_data:
```

Serve per mantenere i dati anche dopo un `docker compose down`.

---

## **6. Come funziona la rete in Compose**

Compose crea automaticamente **una rete privata** per tutti i servizi del file.
I container si vedono per **nome del servizio**, come se fosse un DNS.

| Servizio | Nome DNS interno | Porta interna |
| -------- | ---------------- | ------------- |
| qdrant   | `qdrant`         | 6333          |
| backend  | `backend`        | 8080          |
| ui       | `ui`             | 8501          |

Quindi nel backend puoi scrivere:

```python
client = QdrantClient(host="qdrant", port=6333)
```

e non serve l’IP.

---

## **7. Comandi principali**

| Comando                            | Descrizione                         |
| ---------------------------------- | ----------------------------------- |
| `docker compose up -d`             | avvia tutti i servizi in background |
| `docker compose ps`                | mostra i container attivi           |
| `docker compose logs -f backend`   | visualizza i log del backend        |
| `docker compose exec backend bash` | entra nel container backend         |
| `docker compose down`              | ferma e rimuove tutto               |
| `docker compose build backend`     | ricostruisce solo il backend        |

---

## **8. Benefici concreti per progetti AI**

| Problema                      | Soluzione con Compose                           |
| ----------------------------- | ----------------------------------------------- |
| Setup manuale lungo           | Un solo comando `docker compose up`             |
| Gestione reti e nomi          | DNS interno automatico (`backend`, `qdrant`)    |
| Dati persi a ogni restart     | Volumi persistenti gestiti                      |
| Avvio non sincronizzato       | `depends_on` + `healthcheck`                    |
| Ambiente locale riproducibile | Tutto descritto in YAML, condivisibile nel repo |

---

## **9. Esercizio pratico**

1. Crea la struttura:

   ```
   project/
   ├─ backend/ (con Dockerfile e app.py)
   ├─ ui/ (Streamlit)
   ├─ docker-compose.yml
   └─ config/
   ```
2. Copia il file Compose di esempio.
3. Avvia:

   ```bash
   docker compose up -d
   ```
4. Apri la UI su `localhost:8501` e verifica che si connetta al backend (che parla con Qdrant).

---

## **10. In sintesi**

* Docker Compose è il **collante** dei container.
* Descrive tutto in un unico file: servizi, reti, volumi, porte, variabili.
* Ti permette di **avviare uno stack AI completo con un solo comando**.
* È lo standard per orchestrare ambienti **CrewAI + Qdrant + UI** in locale o in CI/CD.

---




# 3.2 — Dockerfile & Compose *production-grade* per AI

## A) Dockerfile “da produzione” (CPU)

Obiettivi: immagine piccola, build riproducibile, niente tool di build nello stage finale, utente non-root, log immediati.

```dockerfile
# syntax=docker/dockerfile:1.7

############################
# STAGE 1 — builder
############################
FROM python:3.11-slim AS builder
WORKDIR /build

# Dipendenze minime per build (solo qui)
RUN apt-get update && apt-get install -y --no-install-recommends \
      build-essential git curl ca-certificates \
  && rm -rf /var/lib/apt/lists/*

# Copia solo i requirements per massimizzare la cache
COPY requirements.txt .

# Prepara wheelhouse (cache pip con BuildKit)
RUN --mount=type=cache,target=/root/.cache/pip \
    pip install --upgrade pip wheel \
 && pip wheel --no-deps --no-cache-dir -r requirements.txt -w /wheels

############################
# STAGE 2 — runtime
############################
FROM python:3.11-slim AS runtime
ENV PYTHONUNBUFFERED=1 PYTHONDONTWRITEBYTECODE=1 PIP_DISABLE_PIP_VERSION_CHECK=1

# Utente non-root
RUN useradd -m -u 10001 appuser
WORKDIR /app

# Installa SOLO dalle wheel precompilate (niente compilers in runtime)
COPY --from=builder /wheels /wheels
COPY requirements.txt .
RUN --mount=type=cache,target=/root/.cache/pip \
    pip install --no-index --find-links=/wheels -r requirements.txt \
 && rm -rf /wheels

# Copia codice
COPY . .
RUN chown -R appuser:appuser /app
USER appuser

EXPOSE 8080
# Healthcheck leggero (se l’app espone /health)
# HEALTHCHECK --interval=30s --timeout=3s --retries=3 CMD curl -fsS http://localhost:8080/health || exit 1

CMD ["uvicorn","app.main:app","--host","0.0.0.0","--port","8080"]
```

**.dockerignore (essenziale)**

```
.git
__pycache__/
*.pyc
*.pyo
*.pyd
.env
venv/
models/
data/
dist/
build/
node_modules/
```

> Per AI: **non** inserire modelli/weights nell’immagine → monta la cache HF come volume.

---

## B) Dockerfile GPU (NVIDIA) “runtime-only”

Quando ti serve CUDA, usa base **runtime**; se compili estensioni, usa **devel** solo nel builder:

```dockerfile
# syntax=docker/dockerfile:1.7
FROM pytorch/pytorch:2.3.0-cuda12.1-cudnn8-devel AS builder
WORKDIR /build
COPY requirements.txt .
RUN --mount=type=cache,target=/root/.cache/pip \
    pip install --upgrade pip wheel \
 && pip wheel --no-deps --no-cache-dir -r requirements.txt -w /wheels

FROM pytorch/pytorch:2.3.0-cuda12.1-cudnn8-runtime AS runtime
ENV PYTHONUNBUFFERED=1
WORKDIR /app
COPY --from=builder /wheels /wheels
COPY requirements.txt .
RUN --mount=type=cache,target=/root/.cache/pip \
    pip install --no-index --find-links=/wheels -r requirements.txt \
 && rm -rf /wheels
COPY . .
EXPOSE 8080
CMD ["uvicorn","app.main:app","--host","0.0.0.0","--port","8080"]
```

> Host: driver NVIDIA + NVIDIA Container Toolkit. A runtime usa `--gpus all`.

---

## C) Compose “da produzione”: robustezza, ordine di avvio, limiti risorse

### docker-compose.yml (CPU-only)

```yaml
services:
  qdrant:
    image: qdrant/qdrant:v1.10.0
    volumes:
      - qdrant_data:/qdrant/storage
    # NON esporre in prod se non serve
    # ports: ["6333:6333"]
    healthcheck:
      test: ["CMD", "wget", "-qO-", "http://localhost:6333/healthz"]
      interval: 10s
      timeout: 3s
      retries: 5
    restart: unless-stopped

  backend:
    image: myorg/crewai-backend:prod   # oppure build: ./backend
    depends_on:
      qdrant:
        condition: service_healthy
    environment:
      QDRANT_HOST: qdrant
      QDRANT_PORT: "6333"
      # chiavi via env_file o secrets manager
    ports: ["8080:8080"]     # esponi solo ciò che serve
    volumes:
      - ./config:/app/config:ro
      - ./models:/root/.cache/huggingface
    deploy:
      resources:
        limits:
          cpus: "2.0"
          memory: "4g"
    restart: unless-stopped

  ui:
    image: myorg/streamlit-ui:prod     # oppure build: ./ui
    depends_on:
      - backend
    environment:
      API_BASE_URL: http://backend:8080
    ports: ["8501:8501"]
    restart: unless-stopped

volumes:
  qdrant_data:
```

### Varianti utili

* **GPU profile (solo dove supportato)**:

  ```yaml
  services:
    backend-gpu:
      image: myorg/crewai-backend:cuda
      deploy:
        resources:
          reservations:
            devices:
              - capabilities: ["gpu"]
      # ports/volumes/environment come sopra
      profiles: ["gpu"]
  ```

  Avvio: `docker compose --profile gpu up -d`

* **env_file** per caricare variabili:

  ```yaml
  backend:
    env_file: [.env]
  ```

  > Non committare `.env`. Usa vault o secret manager in prod.

* **Reti dedicate (opzionale)**:

  ```yaml
  networks:
    private:
  services:
    qdrant:  { networks: [private] }
    backend: { networks: [private] }
    ui:      { networks: [private] }
  ```

---

## D) Strategie di rebuild selettivo & cicli di sviluppo

* Ricostruire solo un servizio:

  ```bash
  docker compose build backend && docker compose up -d backend
  ```
* Ricostruire e rialzare tutto:

  ```bash
  docker compose up -d --build
  ```
* Log mirati e shell:

  ```bash
  docker compose logs -f backend
  docker compose exec backend bash
  ```
* **Sviluppo hot-reload (solo dev)**: monta il codice

  ```yaml
  backend:
    build: ./backend
    volumes:
      - ./backend:/app
  ```

  In produzione rimuovi questo bind mount: usa l’immagine immutabile.

---

## E) Healthcheck “seri” (FastAPI)

Aggiungi un endpoint `GET /health` nel backend (risposta rapida, no DB pesanti).
Compose:

```yaml
backend:
  # ...
  healthcheck:
    test: ["CMD", "wget", "-qO-", "http://localhost:8080/health"]
    interval: 10s
    timeout: 3s
    retries: 5
```

E usa `depends_on: condition: service_healthy` per orchestrare l’avvio corretto con Qdrant.

---

## F) Sicurezza minima

* **Utente non-root** nello stage runtime (già visto nei Dockerfile).
* Non esporre servizi interni (Qdrant, Postgres) se non necessario.
* Monta config **read-only** (`:ro`).
* Nessuna chiave nel Dockerfile → passa via `env_file` o secret manager.
* Aggiorna regolarmente immagini base; valuta scanner (Trivy/Docker Scout) in CI.

---

## G) Performance & costi immagine

* `.dockerignore` curato (niente `models/`, `data/`, `venv/`, `.env`).
* **Multi-stage** + wheelhouse: runtime snello.
* **Cache pip BuildKit**: `--mount=type=cache,target=/root/.cache/pip`.
* Per GPU: **stage builder = devel**, **stage finale = runtime**.
* **Modelli HF fuori dall’immagine** → volume `./models:/root/.cache/huggingface`.

---

## H) Mini-checklist *prod-ready*

* [ ] Dockerfile multi-stage, runtime senza tool di build
* [ ] Utente non-root, `PYTHONUNBUFFERED=1`
* [ ] `.dockerignore` completo
* [ ] Healthcheck backend & Qdrant, `depends_on` con `service_healthy`
* [ ] Volumi persistenti (Qdrant), cache HF montata
* [ ] Porte esposte solo dove serve
* [ ] Limiti risorse (CPU/MEM) e profili GPU se necessari
* [ ] Segreti via `env_file`/secret manager (mai nel Dockerfile)
* [ ] Strategie di rebuild selettivo (`compose build <svc>`)

---



